<a href="https://colab.research.google.com/github/reservamos/colabs/blob/main/cuadramos_base.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Conexión

In [ ]:
#@title Conectarse a Google Drive
#@markdown Hacer previamente un acceso directo a la carpeta de reportes-mensuales en el drive propio.

from google.colab import drive

drive.mount('/content/drive', force_remount=True)

MessageError: ignored

# 1. General

In [ ]:
#@title 1.1 Leer reportes en la carpeta de reportes-mensuales de Google Drive
mes = "07" #@param ["01", "02", "03", "04", "05", "06", "07", "08", "09", "10", "11", "12"]
año = "2024" #@param ["2024", "2023", "2022", "2025"]

#@markdown [reporte compras](http://reservamos-metabase.us-west-2.elasticbeanstalk.com/question/1456)
nombre_reporte_de_compras = "julio24-compras" #@param {type:"string"}
#@markdown [reporte bus](http://reservamos-metabase.us-west-2.elasticbeanstalk.com/question/1224?&tickets_state=payed&tickets_state=canceled)
nombre_reporte_de_boletos_bus = "julio24-boletos-bus" #@param {type:"string"}
#@markdown [reporte aéreo](http://reservamos-metabase.us-west-2.elasticbeanstalk.com/question/1232?&tickets_state=payed&tickets_state=canceled)
nombre_reporte_de_boletos_avion = "julio24-boletos-avion" #@param {type:"string"}

#@markdown [reporte adyen](https://colab.research.google.com/drive/1fyDcFcPGswliuo84WoWMj5Ej77msW9sn?usp=sharing)
nombre_reporte_adyen = "julio24-adyen" #@param {type:"string"}
#@markdown reporte conekta
nombre_reporte_conekta = "julio24-conekta" #@param {type:"string"}
#@markdown reporte paypal
nombre_reporte_paypal = "julio24-paypal" #@param {type:"string"}
#@markdown reporte openpay
nombre_reporte_openpay = "julio24-openpay" #@param {type:"string"}
#@markdown [reporte monedero](https://data.reservamos.mx/question/1528)
nombre_reporte_monedero = "julio24-monedero" #@param {type:"string"}


from google.colab import files
import pandas as pd
import numpy as np
import os
os.chdir("/content/drive/MyDrive/reportes-mensuales")

linebr = '\r\n'
cdf = pd.read_csv(f'{nombre_reporte_de_compras}.csv')
bdf = pd.read_csv(f'{nombre_reporte_de_boletos_bus}.csv')
adf = pd.read_csv(f'{nombre_reporte_de_boletos_avion}.csv')
adf = pd.read_csv(f'{nombre_reporte_de_boletos_avion}.csv')

tdf = pd.concat([bdf, adf])

adyen_df = pd.read_csv(f'{nombre_reporte_adyen}.csv')
conekta_df = pd.read_csv(f'{nombre_reporte_conekta}.csv')
paypal_df = pd.read_csv(f'{nombre_reporte_paypal}.csv')
openpay_df = pd.read_csv(f'{nombre_reporte_openpay}.csv')

wallet_df = pd.read_csv(f'{nombre_reporte_monedero}.csv')
bus_transporters_df = pd.read_csv('bus-transporters.csv')

print('Archivos cargados')

FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/reportes-mensuales'

In [ ]:
#@title 1.2 Filtrado y ordenado del reporte de compras.

def month(string):
  if len(str(string)) < 2:
    return "0" + str(string)
  return str(string)

def df_counter(df):
  print('-> ' + "{:0,.0f}".format(df.shape[0]) + ' registros.')

def fmt_curr(number):
  return '${:0,.2f}'.format(number)

def fmt_num(number):
  return '{:0,.0f}'.format(number)

def fmt_dnum(number):
  return '{:0,.2f}'.format(number)

def diff_print(num1, num2):
  print(f'DIFERENCIA = {fmt_curr(num1 - num2)}{linebr}procesador: {fmt_curr(num1)} - reservamos: {fmt_curr(num2)}')

def compare_cols(col1, col2):
  not_found_elements = []
  for x in col1:
    if x in col2.values:
      continue
    else:
      not_found_elements.append(x)
  return np.unique(not_found_elements)

def create_payment(row):
    if pd.isnull(row['payment_engine']):
        return row['payment_type']
    else:
        return row['payment_engine']

basic_groupby_columns = ['purchase_state', 'payment_status', 'payment_method']
basic_agg_columns = {'purchase_total':'sum','payment_total':'sum'}
basic_currency_columns = list(basic_agg_columns.keys())
extended_agg_columns = {**basic_agg_columns, 'trip_total':'sum','ancillaries_total':'sum','discounts_total':'sum'}
ext_currency_columns = list(extended_agg_columns.keys())
purchase_columns = {'purchase_total':'sum', 'payment_total':'sum', 'trip_total':'sum', 'ancillaries_total':'sum', 'discounts_total':'sum', 'markup':'sum', 'service_level':'sum', 'insurance':'sum', 'wallet':'sum', 'viajamas':'sum', 'interest':'sum', 'commission':'sum', 'discount':'sum'}
purchase_columns_list = list(purchase_columns.keys())

# sorts purchases report by id
sorted_purchases = cdf.sort_values(by=['purchase_id'])

# filters only purchases paid on selected month or purchases with nopayment
purchases1 = sorted_purchases.query(f"paid_at_mx >= '{año}-{month(mes)}-01' and paid_at_mx < '{año}-{month(int(mes)+1)}-01' or payment_status == 'nopayment'")

# sorts purchases by paid date
purchases2 = purchases1.sort_values(by=['paid_at_mx'])

#add calculated columns
purchases2.loc[:, "trip_total"] = purchases2.price + purchases2.taxes + purchases2.service_fee - purchases2.provider_discount
purchases2.loc[:, "ancillaries_total"] = purchases2.markup + purchases2.insurance + purchases2.service_level + purchases2.interest
purchases2.loc[:, "discounts_total"] = purchases2.wallet + purchases2.viajamas + purchases2.discount
purchases2['payment_method'] = purchases2.apply(create_payment, axis=1)

# drops duplicates with failed and charged payments, keeps only charged row, this happens due to query validation (payment.pdt is not null OR payment.status = 'charged')
purchases3 = purchases2.drop_duplicates(subset=['purchase_id', 'purchase_state'], keep='last')

df_counter(purchases3)

first_id = purchases3.iloc[0]['purchase_id']
last_id = purchases3.iloc[-1]['purchase_id']
print('Primer id de compra: ' + str(first_id) + ', último id: ' + str(last_id))

payment_status_before = { 'charged': 'charged', 'attempt': 'attempt', 'pending': 'failed', 'canceled': 'rejected', 'fully_refunded': 'charged', 'partially_refunded': 'charged', 'suspicious': 'charged', 'chargeback': 'charged', 'failed': 'failed', 'nopayment': 'nopayment', 'unsafe': 'unsafe'}
purchases3.loc[:, "payment_status_before"] = purchases3.payment_status.apply(lambda x: payment_status_before[x])
payment_status_final = { 'charged': 'charged', 'attempt': 'attempt', 'pending': 'failed', 'canceled': 'rejected', 'fully_refunded': 'refund', 'partially_refunded': 'refund', 'suspicious': 'charged', 'chargeback': 'chargeback', 'failed': 'failed', 'nopayment': 'pre_charged', 'unsafe': 'unsafe'}
purchases3.loc[:, "payment_status_final"] = purchases3.payment_status.apply(lambda x: payment_status_final[x])

income_purchases = purchases3.query("payment_status_final == 'charged'")
#income_purchases.loc[:, 'purchase_type'] = 'aceptadas'
#income_purchases.loc[income_purchases.payment_status == "nopayment", "total"] = income_purchases.loc[:,'purchase_total']
#income_purchases.loc[income_purchases.payment_status == "nopayment", "paid_at_mx"] = income_purchases.loc[:,'completed_at_mx']

loss_purchases = purchases3.query("payment_status_final == 'failed'")
#loss_purchases.loc[:, 'purchase_type'] = 'fallidas'
#loss_purchases.loc[loss_purchases.payment_status == "nopayment", "payment_total"] = loss_purchases.loc[:,'purchase_total']

# income_purchases.to_csv('df.csv', encoding = 'utf-8-sig')
# files.download('df.csv')

In [ ]:
#@title 1.3 Filtrado y ordenado del reporte de boletos.

# sorts tickets report by id
sorted_tickets = tdf.sort_values(by=['ID Compra'])

# drops duplicates with same ticket id
tickets1 = sorted_tickets.drop_duplicates(subset=['ID Boleto'], keep='last')

#adds one column of service_levels that summarizes existents
tickets1['Nds boleto'] =  np.where(tickets1['Nds x boleto (>4jul)'].isnull(), tickets1['Nds por boleto'], tickets1['Nds x boleto (>4jul)'])

print('No. de boletos: ' + str(tickets1.shape[0]))
purchases_on_tickets = np.unique(tickets1['ID Compra'].values)
print('No. de compras: ' + str(len(purchases_on_tickets)))

first = tickets1.iloc[0]['ID Boleto']
last = tickets1.iloc[-1]['ID Boleto']
print('Primer id de boleto: ' + str(first) + ', último id: ' + str(last))

tickets_on_purchases = purchases3.query("ticket_count > 0")
purchases_on_purchases = np.unique(tickets_on_purchases['purchase_id'].values)
print('No. de compras en el reporte de compras: ' + str(len(purchases_on_purchases)) + linebr)

print('Ids compras diferentes: ' + str(set(purchases_on_purchases) - set(purchases_on_tickets)) + ' (punto 3.2)')

In [ ]:
#@title 1.4 Diferencias en montos en reporte de compras.
#@markdown Diferencias de centavos entre el total del pago y el total de la compra guardados en la base de datos.
diferencias_en_campos = "desglose" #@param ["desglose", "registros"]

purchases_diffs = purchases3.query("purchase_total != payment_total and purchase_total > 0 and payment_status != 'nopayment'")
purchases_diffs2 = purchases_diffs[['purchase_id', 'purchase_state', 'payment_status', 'payment_method', 'vendor', 'channel', 'purchase_total', 'payment_total', 'completed_at_mx']]

df_counter(purchases_diffs2)

purchases_diffs_breakdown = purchases_diffs2.groupby(['vendor', 'channel'] + basic_groupby_columns, dropna=False).agg({'channel':'count', **basic_agg_columns})
purchases_diffs_breakdown['diff'] = purchases_diffs_breakdown.payment_total - purchases_diffs_breakdown.purchase_total
diff_print(purchases_diffs_breakdown['purchase_total'].sum(), purchases_diffs_breakdown['payment_total'].sum())
purchases_diffs_breakdown[basic_currency_columns] = purchases_diffs_breakdown[basic_currency_columns].apply(lambda series: series.apply(lambda value: fmt_dnum(value)) )

purchases_diffs_breakdown.style if diferencias_en_campos == 'desglose' else purchases_diffs2.style

In [ ]:
#@title 1.5 Diferencias en montos entre reporte de compras y boletos.
#@markdown Diferencias de centavos entre el total del boleto en el reporte de boleto y el total del boleto en el reporte de la compra.
#@markdown La comparación se hace tabla de compras contra tabla de boletos. Diferencias en markup son porque en el reporte es una columna calculada por boleto, el que se cobró realmente es el markup de la compra.
diferencias_en_campos = "registros" #@param ["desglose", "registros"]

purchase_trip_totals = purchases3.query("price > 0")
purchase_trip_totals_subset = purchase_trip_totals[['purchase_id', 'purchase_state', 'payment_status', 'payment_method', 'vendor', 'channel', 'ticket_count', 'price', 'taxes', 'service_fee', 'provider_discount', 'service_level', 'markup', 'trip_total']]

tickets_subset = tickets1[['ID Compra', 'Precio', 'IVA', 'Total', 'Descuento', 'Nds boleto', 'Markup']]
tickets_totals_by_purchase = tickets_subset.groupby(['ID Compra'], dropna=False).agg(tickets_count=('ID Compra', 'count'), tickets_total=('Total', 'sum'), tickets_markup=('Markup', 'sum'),  tickets_service_level=('Nds boleto', 'sum')).reset_index()

trip_totals_df = pd.merge(purchase_trip_totals_subset, tickets_totals_by_purchase, how='inner', left_on = 'purchase_id', right_on = 'ID Compra')

trip_totals_df['total_diff'] = trip_totals_df.trip_total - trip_totals_df.tickets_total
trip_totals_df['markup_diff'] = trip_totals_df.markup - trip_totals_df.tickets_markup
trip_totals_df['service_level_diff'] = trip_totals_df.service_level - trip_totals_df.tickets_service_level

tickets_totals_breakdown = trip_totals_df.query('total_diff != 0 or markup_diff != 0 or service_level_diff != 0')
tickets_totals_breakdown_subset = tickets_totals_breakdown[['purchase_id', 'purchase_state', 'payment_status', 'payment_method', 'vendor', 'channel', 'ticket_count', 'tickets_count', 'trip_total', 'tickets_total', 'total_diff', 'service_level', 'tickets_service_level', 'service_level_diff', 'markup', 'tickets_markup', 'markup_diff']]

tickets_totals_breakdown2 = tickets_totals_breakdown_subset.groupby(['purchase_state', 'payment_status'], dropna=False).agg(count=('payment_status','count'), purchase_trip_total=('trip_total', 'sum'), tickets_trip_total=('tickets_total', 'sum'), total_diff=('total_diff', 'sum'), purchase_markup=('markup', 'sum'), tickets_markup=('tickets_markup', 'sum'), markup_diff=('markup_diff', 'sum'), purchase_service_level=('service_level', 'sum'), tickets_service_level=('tickets_service_level', 'sum'), service_level_diff=('service_level_diff', 'sum'))
diff_print(tickets_totals_breakdown2['purchase_trip_total'].sum(), tickets_totals_breakdown2['tickets_trip_total'].sum())

diff_columns_names = ['purchase_trip_total', 'tickets_trip_total', 'total_diff', 'purchase_markup', 'tickets_markup', 'markup_diff', 'purchase_service_level', 'tickets_service_level', 'service_level_diff']
tickets_totals_breakdown2[diff_columns_names] = tickets_totals_breakdown2[diff_columns_names].apply(lambda series: series.apply(lambda value: fmt_dnum(value)) )

df_counter(tickets_totals_breakdown_subset)

tickets_totals_breakdown2.style if diferencias_en_campos == 'desglose' else tickets_totals_breakdown_subset.style

# 2. Compras

In [ ]:
#@title 2.1 Cálculo de ingresos en reporte de compras.

full_breakdown = purchases3.groupby(['purchase_state'], dropna=False).agg({'purchase_state': 'count', **purchase_columns})
df_counter(purchases3)

full_breakdown[purchase_columns_list] = full_breakdown[purchase_columns_list].apply(lambda series: series.apply(lambda value: fmt_dnum(value)) )
full_breakdown.style

# to show purchases from breakdown paste 'purchases3.style' on the next line.

In [ ]:
#@title 2.2 Compras por estatus de pago.

df_counter(purchases3)

pstatus_breakdown = purchases3.groupby(['purchase_state', 'payment_status'], dropna=False).agg({'payment_status': 'count', **purchase_columns})
pstatus_breakdown[purchase_columns_list] = pstatus_breakdown[purchase_columns_list].apply(lambda series: series.apply(lambda value: fmt_dnum(value)) )
pstatus_breakdown.style

# to show purchases from breakdown paste 'purchases3.style' on the next line.

In [ ]:
#@title 2.3 Compras por procesador de pago
#@markdown Las nan son las compras forzadas, es decir, que no pasaron por procesador de pagos.

df_counter(purchases3)

engine_breakdown = purchases3.groupby(["payment_engine"], dropna=False).agg({'payment_engine':'count', **purchase_columns})
engine_breakdown[purchase_columns_list] = engine_breakdown[purchase_columns_list].apply(lambda series: series.apply(lambda value: fmt_dnum(value)) )
engine_breakdown.style

# to show purchases from breakdown paste 'purchases3.style' on the next line.

In [ ]:
#@title 2.4 Compras por vendedor y canal.
df_counter(purchases3)

vendor_breakdown = purchases3.groupby(['vendor', 'channel'], dropna=False).agg({'channel':'count', **purchase_columns})
vendor_breakdown[purchase_columns_list] = vendor_breakdown[purchase_columns_list].apply(lambda series: series.apply(lambda value: fmt_dnum(value)) )
vendor_breakdown.style

# to show purchases from breakdown paste 'purchases3.style' on the next line.

In [ ]:
#@title 2.5 Compras con niveles de servicio.
#@markdown

nds_purchases = purchases3.query("service_level > 0")
nds_breakdown = nds_purchases.groupby(['purchase_state'], dropna=False).agg({'purchase_state':'count', **purchase_columns})
nds_breakdown[purchase_columns_list] = nds_breakdown[purchase_columns_list].apply(lambda series: series.apply(lambda value: fmt_dnum(value)) )

df_counter(nds_purchases)
nds_breakdown.style

# to show purchases from breakdown paste 'nds_purchases.style' on the next line.

In [ ]:
#@title 2.6 Compras con seguro.
#@markdown
filtrar_seguro= "desglose" #@param ["desglose", "registros"]

insurance_purchases = purchases3.query("insurance > 0")
insurance_breakdown = insurance_purchases.groupby(['purchase_state'], dropna=False).agg({'purchase_state':'count', **purchase_columns})
insurance_breakdown[purchase_columns_list] = insurance_breakdown[purchase_columns_list].apply(lambda series: series.apply(lambda value: fmt_dnum(value)) )

df_counter(insurance_purchases)
insurance_breakdown.style

insurance_breakdown.style if filtrar_seguro == 'desglose' else insurance_purchases.style
# to show purchases from breakdown paste 'insurance_purchases.style' on the next line.

In [ ]:
#@title 2.7 Compras con monedero.
#@markdown
filtrar_monedero = "desglose" #@param ["desglose", "registros"]

wallet_purchases = purchases3.query("wallet > 0")
wallet_breakdown = wallet_purchases.groupby(['purchase_state'], dropna=False).agg({'purchase_state':'count', **purchase_columns})
wallet_breakdown[purchase_columns_list] = wallet_breakdown[purchase_columns_list].apply(lambda series: series.apply(lambda value: fmt_dnum(value)) )

df_counter(wallet_purchases)
wallet_breakdown.style

wallet_breakdown.style if filtrar_monedero == 'desglose' else wallet_purchases.style
# to show purchases from breakdown paste 'wallet_purchases.style' on the next line.

In [ ]:
#@title 2.8 Compras a meses.
#@markdown
filtrar_meses = "desglose" #@param ["desglose", "registros"]

interest_purchases = purchases3.query("interest > 0")
interest_breakdown = interest_purchases.groupby(['purchase_state'], dropna=False).agg({'purchase_state':'count', **purchase_columns})
interest_breakdown[purchase_columns_list] = interest_breakdown[purchase_columns_list].apply(lambda series: series.apply(lambda value: fmt_dnum(value)) )

df_counter(interest_purchases)
interest_breakdown.style

interest_breakdown.style if filtrar_meses == 'desglose' else interest_purchases.style
# to show purchases from breakdown paste 'interest_purchases.style' on the next line.

In [ ]:
#@title 2.9 Compras con viajamas.
#@markdown
filtrar_viajamas = "desglose" #@param ["desglose", "registros"]

viajamas_purchases = purchases3.query("viajamas > 0")
viajamas_breakdown = viajamas_purchases.groupby(['purchase_state'], dropna=False).agg({'purchase_state':'count', **purchase_columns})
viajamas_breakdown[purchase_columns_list] = viajamas_breakdown[purchase_columns_list].apply(lambda series: series.apply(lambda value: fmt_dnum(value)) )

df_counter(viajamas_purchases)
viajamas_breakdown.style

viajamas_breakdown.style if filtrar_viajamas == 'desglose' else viajamas_purchases.style
# to show purchases from breakdown paste 'viajamas_purchases.style' on the next line.

# Compras con pagos fallidos

In [ ]:
#@title 2*.10 Compras fallidas con fecha de pago.
#@markdown Son todas las compras fallidas, incluídas las duplicadas del 2*.2, es decir, las que estan asociadas a una misma compra, por cada pago fallido cobrado de una misma compra es posible que se cobrara el total de la misma. Se deben checar para ver si procede reembolso.
filtrar_fallidas = "desglose" #@param ["desglose", "registros"]

all_failed_purchases = purchases2.query("payment_status == 'failed'")
df_counter(all_failed_purchases)

failed_breakdown = all_failed_purchases.groupby(basic_groupby_columns, dropna=False).agg({'payment_status':'count', **purchase_columns})
failed_breakdown[purchase_columns_list] = failed_breakdown[purchase_columns_list].apply(lambda series: series.apply(lambda value: fmt_dnum(value)) )

failed_breakdown.style if filtrar_fallidas == 'desglose' else all_failed_purchases.style

In [ ]:
#@title 2*.2 Compras fallidas con fecha de pago, compras con varios intentos de pago.
#@markdown Se incluyen sólo las duplicadas, es decir, las más antiguas pertenecientes a una misma compra que fueron filtradas en el reporte de compras, ya que sólo se dejó el último pago asociado a una compra.
#@markdown No están incluidas en los puntos 2.1 a 2.4, ni en el 5.1 por lo que debe revisarse si se cobraron en el reporte del procesador de pago para considerar su monto en los ingresos.
filtrar_fallidas = "desglose" #@param ["desglose", "registros"]

not_included_purchases_on_filtered_report = compare_cols(all_failed_purchases['payment_token'], purchases3['payment_token'])
purchases6 = all_failed_purchases.loc[all_failed_purchases['payment_token'].isin(not_included_purchases_on_filtered_report)]
df_counter(purchases6)

dups_breakdown = purchases6.groupby(basic_groupby_columns, dropna=False).agg({'payment_status':'count', **purchase_columns})
dups_breakdown[purchase_columns_list] = dups_breakdown[purchase_columns_list].apply(lambda series: series.apply(lambda value: fmt_dnum(value)) )

adyen_dups_not_included = purchases6.query("payment_engine == 'adyen'")
dups_breakdown.style if filtrar_fallidas == 'desglose' else purchases6.style

# 3. Boletos (bus + avión)

In [ ]:
#@title 3.1 Conteo de boletos por proveedor e ingresos.
#@markdown Seleccionar si desea agruparlos por proveedor o por línea:
agrupados_por = "Línea" #@param ["Proveedor", "Línea"]

tickets_columns = {'Total':'sum', 'Nds por boleto':'sum', 'Nds x boleto (>4jul)':'sum', 'Viaja Seguro por boleto':'sum', 'Comisión (sin IVA)': 'sum', 'IVA Comisión': 'sum', 'Markup': 'sum', 'Servicio de línea': 'sum' }
tickets_columns_list = list(tickets_columns.keys())

tickets_breakdown = tickets1.groupby([agrupados_por, 'Estado de compra', 'Estado ante operador'], dropna=False).agg({'Línea':'count', **tickets_columns})

tickets2 = tickets1.query("`Estado de compra` in ('chargeback', 'canceled')")

money_charged = tickets_breakdown['Total'].sum()
money_not_kept = tickets2['Total'].sum()
income = money_charged - money_not_kept

df_counter(tickets1)
print('Dinero cobrado en boletos: ' + "${:0,.2f}".format(money_charged))
print('Dinero que no conservamos: ' + "${:0,.2f}".format(money_not_kept) + '(contracargos y cancelaciones)')
print('Ingreso total en boletos: ' + "${:0,.2f}".format(income))

print('* Total en la tabla es total en boleto, no de la compra')
tickets_breakdown[tickets_columns_list] = tickets_breakdown[tickets_columns_list].apply(lambda series: series.apply(lambda value: fmt_dnum(value)) )
tickets_breakdown.style

In [ ]:
#@title 3.2 Diferencias de compras entre reporte de boletos y reporte de compras.
#@markdown Compras que no aparecen en el reporte de boletos pero sí en el de compras y viceversa.
#@markdown Canceladas a ajustarse a meses anteriores.
#@markdown Boletos creados y/ó completados en el mes anterior pero actualizados (cancelados) en el mes en cuestión, por lo tanto aparecen en el reporte de boletos pero no en el de compras.
#@markdown Aparecen también boletos de compras creadas y pagadas el mes anterior pero completadas dentro del mes, como los boletos fueron creados al completarse la compra dentro del mes en cuestión aparecen en el reporte de boletos pero como el pago fue el mes anterior no aparecen en el reporte de compras. La última columna tiene la fecha de pago de algunas de las compras.
#@markdown Aparecen también compras pagadas dentro del mes pero como fueron completadas hasta el sig mes, los boletos no aparecerán porque se crearon al completarse la compra. Aparecerán en el reporte de boletos del mes siguiente.
diferencias_reportes = "desglose" #@param ["desglose", "registros"]

tickets_subset = pd.DataFrame().assign(purchase_id=tickets1['ID Compra'], purchase_state=tickets1['Estado de compra'], df='boletos', amount=tickets1['Total'], ticket_type=tickets1['Estado ante operador'], date=tickets1['Compra completada CST'], paid_date='N/D')
tickets_subset['date'] = tickets_subset['date'].str[:10]
tickets_by_purchase = tickets_subset.groupby(['purchase_id', 'purchase_state', 'df', 'date', 'paid_date'], dropna=False).agg(ticket_count=('purchase_id', 'count'), amount=('amount', 'sum'), ticket_type=('ticket_type', lambda x: ','.join(np.unique(x)))).reset_index()

purchases_purchase_ids = pd.DataFrame().assign(purchase_id=purchases3['purchase_id'], purchase_state=purchases3['purchase_state'], df='compras', ticket_count=purchases3['ticket_count'], amount=purchases3['price'], ticket_type='N/D', date=purchases3['completed_at_mx'], paid_date=purchases3['paid_at_mx'])
purchases_purchase_ids['date'] = purchases_purchase_ids['date'].str[:10]
purchases_purchase_ids['paid_date'] = purchases_purchase_ids['paid_date'].str[:10]

reports_diffs = pd.concat([tickets_by_purchase, purchases_purchase_ids])
reports_diffs_without_dups = reports_diffs.drop_duplicates(subset=['purchase_id'], keep=False)
reports_diffs_subset = reports_diffs_without_dups[['df', 'purchase_id', 'purchase_state', 'ticket_count', 'ticket_type', 'amount', 'date', 'paid_date']].reset_index(drop=True)
df_counter(reports_diffs_subset)

reports_diff_breakdown = reports_diffs_subset.groupby(['df', 'purchase_state', 'ticket_type'], dropna=False).agg({'ticket_count':'count', 'amount': 'sum'})
reports_diff_breakdown[['amount']] = reports_diff_breakdown[['amount']].apply(lambda series: series.apply(lambda value: fmt_dnum(value)) )

reports_diff_breakdown.style if diferencias_reportes == 'desglose' else reports_diffs_subset.query("purchase_state != 'attempt'").style

In [ ]:
#@title 3.3.1 Boletos de autobús.
#@markdown Ingresos por todos los boletos de autobús vendidos por Reservamos.

bus_tickets = tickets1.query("Tipo == 'BusTicket'")

bus_tickets_breakdown = bus_tickets.groupby(['Estado de compra', 'Estado ante operador'], dropna=False).agg({'Línea':'count', **tickets_columns})

bus_money_charged = bus_tickets_breakdown['Total'].sum()
print('Dinero cobrado en boletos de autobús: ' + "${:0,.2f}".format(bus_money_charged) + linebr)
print('* Total en la tabla es total en boleto, no de la compra.')
print('* Las columnas vienen del boleto, no de la compra.')
bus_tickets_breakdown[tickets_columns_list] = bus_tickets_breakdown[tickets_columns_list].apply(lambda series: series.apply(lambda value: fmt_dnum(value)) )
bus_tickets_breakdown.style

In [ ]:
#@title 3.3.2 Boletos de autobús sin multicarrier.
#@markdown Ingresos por todos los boletos de autobús vendidos por Reservamos sin boletos pertenecientes a compras multicarrier.

bus_tickets_without_mix = tickets1.query("Tipo == 'BusTicket' and Multicarrier == False")

bus_tickets_breakdown2 = bus_tickets_without_mix.groupby(['Estado de compra', 'Estado ante operador'], dropna=False).agg({'Línea':'count', **tickets_columns})

bus_money_charged2 = bus_tickets_breakdown2['Total'].sum()
print('Dinero cobrado en boletos de autobús: ' + "${:0,.2f}".format(bus_money_charged2) + linebr)

bus_tickets_purchases_ids = bus_tickets_without_mix['ID Compra'].values
bus_tickets_purchases = purchases3.loc[purchases3['purchase_id'].isin(bus_tickets_purchases_ids)]

bus_purchase_total = bus_tickets_purchases['purchase_total'].sum()
bus_payment_total = bus_tickets_purchases['payment_total'].sum()
bus_markup = bus_tickets_purchases['markup'].sum()
bus_nds = bus_tickets_purchases['service_level'].sum()
bus_insurance = bus_tickets_purchases['insurance'].sum()
bus_comm = bus_tickets_purchases['commission'].sum()
print('Ingreso total en compras de boletos de autobús: ' + "${:0,.2f}".format(bus_purchase_total))
print('Ingreso total en pagos de boletos de autobús: ' + "${:0,.2f}".format(bus_payment_total))
print('Markup de compras de boletos de autobús: ' + "${:0,.2f}".format(bus_markup))
print('NDS de compras de boletos de autobús: ' + "${:0,.2f}".format(bus_nds))
print('Seguro de compras de boletos de autobús: ' + "${:0,.2f}".format(bus_insurance))
print('Comisión de compras de boletos de autobús: ' + "${:0,.2f}".format(bus_comm))

bus_tickets_breakdown2[tickets_columns_list] = bus_tickets_breakdown2[tickets_columns_list].apply(lambda series: series.apply(lambda value: fmt_dnum(value)) )
bus_tickets_breakdown2.style

In [ ]:
#@title 3.4.1 Boletos de avión.
#@markdown

air_tickets = tickets1.query("Tipo == 'AirplaneTicket'")

air_tickets_breakdown = air_tickets.groupby(['Estado de compra', 'Estado ante operador'], dropna=False).agg({'Línea':'count', **tickets_columns})

air_money_charged = air_tickets_breakdown['Total'].sum()
print('Dinero cobrado en boletos aéreos: ' + "${:0,.2f}".format(air_money_charged))

print('* Total en la tabla es total en boleto, no de la compra')
air_tickets_breakdown[tickets_columns_list] = air_tickets_breakdown[tickets_columns_list].apply(lambda series: series.apply(lambda value: fmt_dnum(value)) )
air_tickets_breakdown.style

In [ ]:
#@title 3.4.2 Boletos de avión sin multicarrier.
#@markdown Ingresos por todos los boletos de avión vendidos por Reservamos sin boletos pertenecientes a compras multicarrier.

air_tickets_without_mix = tickets1.query("Tipo == 'AirplaneTicket' and Multicarrier == False")

air_tickets_breakdown2 = air_tickets_without_mix.groupby(['Estado de compra', 'Estado ante operador'], dropna=False).agg({'Línea':'count', **tickets_columns})

air_money_charged2 = air_tickets_breakdown2['Total'].sum()
print('Dinero cobrado en boletos de avión: ' + "${:0,.2f}".format(air_money_charged2) + linebr)

air_tickets_purchases_ids = air_tickets_without_mix['ID Compra'].values
air_tickets_purchases = purchases3.loc[purchases3['purchase_id'].isin(bus_tickets_purchases_ids)]

air_purchase_total = air_tickets_purchases['purchase_total'].sum()
air_payment_total = air_tickets_purchases['payment_total'].sum()
air_markup = air_tickets_purchases['markup'].sum()
air_nds = air_tickets_purchases['service_level'].sum()
air_insurance = air_tickets_purchases['insurance'].sum()
air_comm = air_tickets_purchases['commission'].sum()
print('Ingreso total en compras de boletos de avión: ' + "${:0,.2f}".format(air_purchase_total))
print('Ingreso total en pagos de boletos de avión: ' + "${:0,.2f}".format(air_payment_total))
print('Markup de compras de boletos de avión: ' + "${:0,.2f}".format(air_markup))
print('NDS de compras de boletos de avión: ' + "${:0,.2f}".format(air_nds))
print('Seguro de compras de boletos de avión: ' + "${:0,.2f}".format(air_insurance))
print('Comisión de compras de boletos de avión: ' + "${:0,.2f}".format(air_comm))

air_tickets_breakdown2[tickets_columns_list] = air_tickets_breakdown2[tickets_columns_list].apply(lambda series: series.apply(lambda value: fmt_dnum(value)) )
air_tickets_breakdown2.style

In [ ]:
#@title 3.5 Top 20 rutas vendidas del mes.
#@markdown

slug_tickets_breakdown = tickets1.groupby(['Cd Origen', 'Cd Destino'], dropna=False).agg(count=('Código destino', 'count')).sort_values(by=['count'], ascending=False).head(20)
slug_tickets_breakdown.style

# 4. Multicarrier

In [ ]:
#@title 4.1 Compras multicarrier.
mix_tickets = tickets1.query("Multicarrier == True")
mix_tickets_ids = pd.unique(mix_tickets['ID Compra'].values)

mix_purchases = purchases3.loc[purchases3['purchase_id'].isin(mix_tickets_ids)]
df_counter(mix_purchases)

mix_income = mix_purchases['purchase_total'].sum()
print('Dinero ingresado por compras multicarrier: ' + "${:0,.2f}".format(mix_income))

mix_purchases_breakdown = mix_purchases.groupby(['purchase_state', 'payment_status'], dropna=False).agg({'payment_status': 'count', **purchase_columns})
mix_purchases_breakdown[purchase_columns_list] = mix_purchases_breakdown[purchase_columns_list].apply(lambda series: series.apply(lambda value: fmt_dnum(value)) )
mix_purchases_breakdown.style


In [ ]:
#@title 4.2 Boletos multicarrier.
#@markdown Boletos parte de una compra multicarrier. Seleccionar si desea agruparlos por proveedor o por línea:
boletos_multicarrier_por = "Proveedor" #@param ["Proveedor", "Línea"]

print(mix_tickets.shape[0])

mix_tickets_income = mix_tickets['Total'].sum()
print('Dinero ingresado por boletos multicarrier: ' + "${:0,.2f}".format(mix_tickets_income))
bus_mix_tickets_income = mix_tickets.query("Tipo == 'BusTicket'")['Total'].sum()
print('Ingreso por boletos multicarrier de autobús: ' + "${:0,.2f}".format(bus_mix_tickets_income))
air_mix_tickets_income = mix_tickets.query("Tipo == 'AirplaneTicket'")['Total'].sum()
print('Ingreso por boletos multicarrier de autobús: ' + "${:0,.2f}".format(air_mix_tickets_income))

mix_purchases_breakdown = mix_tickets.groupby([boletos_multicarrier_por, 'Estado de compra', 'Estado ante operador'], dropna=False).agg({'Línea':'count', **tickets_columns})
mix_purchases_breakdown[tickets_columns_list] = mix_purchases_breakdown[tickets_columns_list].apply(lambda series: series.apply(lambda value: fmt_dnum(value)) )
mix_purchases_breakdown.style

# 5. Adyen

In [ ]:
#@title 5.1 Pagos en Reservamos con Adyen.
#@markdown Todos los pagos de Adyen en Reservamos.
filtrar_adyen = "desglose" #@param ["desglose", "registros"]

# filters only purchases paid by adyen.
adyen_purchases = purchases3.query("payment_engine == 'adyen'")

df_counter(adyen_purchases)

adyen_breakdown = adyen_purchases.groupby(basic_groupby_columns, dropna=False).agg({'payment_engine':'count', **basic_agg_columns})
adyen_breakdown[basic_currency_columns] = adyen_breakdown[basic_currency_columns].apply(lambda series: series.apply(lambda value: fmt_dnum(value)))
adyen_breakdown.style

adyen_breakdown.style if filtrar_adyen == 'desglose' else adyen_purchases.style

In [ ]:
#@title 5.2 Pagos en Adyen.
#@markdown Todos los pagos en el batch del mes.
filtrar_adyen= "desglose" #@param ["desglose", "registros"]

adyen_df['Creation Month'] = adyen_df['Creation Date'].str[5:7]
adyen_df['Gross Credit (GC)'] = adyen_df['Gross Credit (GC)'].replace(',', '', regex=True).astype(float)
df_counter(adyen_df)

filtered_adyen_df = adyen_df.query(f"`Creation Date` < '{año}-{month(int(mes)+1)}-01'")

adyen_breakdown = filtered_adyen_df.groupby(['Creation Month', 'Type'], dropna=False).agg(count=('Type','count'), gross_credit=('Gross Credit (GC)','sum'), gross_debit=('Gross Debit (GC)', 'sum'))
adyen_breakdown[['gross_credit', 'gross_debit']] = adyen_breakdown[['gross_credit', 'gross_debit']].apply(lambda series: series.apply(lambda value: fmt_dnum(value)) )

monthly_adyen_df = adyen_df.sort_values(by=['Creation Date']).query(f"`Creation Date` >= '{año}-{month(mes)}-01' and `Creation Date` < '{año}-{month(int(mes)+1)}-01' and Type == 'Settled'").drop_duplicates(['Psp Reference'],keep='last')
all_monthly_adyen_df = adyen_df.sort_values(by=['Creation Date']).query(f"`Creation Date` >= '{año}-{month(mes)}-01' and `Creation Date` < '{año}-{month(int(mes)+1)}-01'")

print("No. transacciones en el mes:")
df_counter(monthly_adyen_df)

adyen_breakdown.style if filtrar_adyen == 'desglose' else filtered_adyen_df.style

In [ ]:
#@title 5.3 Diferencias de pagos entre Reservamos y Adyen en el mes.
#@markdown Pagos de compras aceptadas que no aparecen en el reporte de Adyen pero sí en el de Reservamos y viceversa en el mes.<br> Diferencias se guardan en un reporte en drive en el punto 11. <br>
#@markdown A veces la mayoría de los pagos que no aparecen en el reporte de Adyen suelen ser de amex. <br>
#@markdown Las referencias psp pueden pertenecer a compras de un mes anterior o a compras completadas pero pagos pendientes de pasar a estatus settled. <br>
#@markdown Las referencias PSP en el reporte de Adyen que no aparecen en el de compras, usualmente son compras donde el pago falló, se reintentó y existe un pago exitoso asociado a la compra, las que tienen estatus completed parecen ser compras con doble cobro y las que tienen estatus attempt son compras fallidas cobradas. Son las compras con pagos duplicados debido a múltiples intentos, por lo tanto tienen pagos que no están considerados en el total del reporte de compras. Totales en el punto 2.5.1, son los que deben considerarse tmb como ingresos.
diferencias_adyen = "desglose" #@param ["desglose", "registros"]

income_purchases_adyen = income_purchases.query("payment_engine == 'adyen'")

adyen_tokens = pd.DataFrame().assign(tokens=monthly_adyen_df['Psp Reference'], engine_status=monthly_adyen_df['Type'], amount=monthly_adyen_df['Gross Credit (GC)'], method=monthly_adyen_df['Payment Method'], date=monthly_adyen_df['Creation Date'], df='only_on_adyen', creation_month=monthly_adyen_df['Creation Month'])

reservamos_adyen_tokens = pd.DataFrame().assign(tokens=income_purchases_adyen['payment_token'], payment_status=income_purchases_adyen['payment_status'], purchase_state=income_purchases_adyen['purchase_state'], amount=income_purchases_adyen['payment_total'], method=income_purchases_adyen['card_brand'], date=income_purchases_adyen['paid_at_mx'], df='only_on_reservamos', purchase_id=income_purchases_adyen['purchase_id'], creation_month=income_purchases_adyen['paid_at_mx'].str[5:7], ticket_count=income_purchases_adyen['ticket_count'],  engine='adyen')

adyen_diffs = pd.concat([adyen_tokens, reservamos_adyen_tokens]).drop_duplicates(subset=['tokens'], keep=False)
adyen_diffs_subset = adyen_diffs[['df', 'tokens', 'payment_status', 'amount', 'method', 'date', 'engine_status', 'purchase_id', 'ticket_count', 'engine', 'purchase_state']].reset_index(drop=True)
df_counter(adyen_diffs_subset)

adyen_diffs_subset.to_csv('adyen_diffs_subset.csv', index=False, encoding='utf-8-sig')

adyen_diff_breakdown = adyen_diffs_subset.groupby(['df', 'purchase_state', 'payment_status', 'engine_status', 'method'], dropna=False).agg(count=('method','count'), amount=('amount','sum'))

adyen_diff_btotal = adyen_diff_breakdown.query("df == 'only_on_adyen'")['amount'].sum()
reservamos_diff_btotal = adyen_diff_breakdown.query("df == 'only_on_reservamos'")['amount'].sum()
print('only_on_adyen: ' + str(fmt_curr(adyen_diff_btotal)))
print('only_on_reservamos: ' + str(fmt_curr(reservamos_diff_btotal)))
print('diferencia: ' + str(fmt_curr(reservamos_diff_btotal - adyen_diff_btotal)))

adyen_diff_breakdown[['amount']] = adyen_diff_breakdown[['amount']].apply(lambda series: series.apply(lambda value: fmt_dnum(value)) )

adyen_next_month_charges = adyen_diffs_subset.query("df == 'only_on_reservamos'")

adyen_diff_breakdown.style if diferencias_adyen == 'desglose' else adyen_diffs_subset.style

In [ ]:
#@title 5.4 Transacciones fallidas en Reservamos de Adyen.
#@markdown Todos los pagos fallidos de Adyen en nuestro sistema. <br> Pagos con Type Settled son pagos que Adyen sí cobró. <br> Las Type nan no fueron encontradas en el reporte de Adyen, por eso no tienen tipo.<br> Las Compras para checar se guardan en un reporte en drive en el punto 11.
fallidas_en_procesador = "desglose" #@param ["desglose", "registros"]

adyen_failed = purchases2.query("payment_status == 'failed' and payment_engine == 'adyen'")
adyen_failed2 = pd.merge(adyen_failed, adyen_df[['Psp Reference','Type']], how='left', left_on=['payment_token'], right_on=['Psp Reference'])

df_counter(adyen_failed)

failed_adyen_breakdown1 = adyen_failed2.groupby(['purchase_state', 'payment_status', 'Type'], dropna=False).agg({'payment_status':'count', **basic_agg_columns})
failed_adyen_breakdown2 = adyen_failed2.groupby(['purchase_id', 'purchase_state', 'payment_status', 'Type'], dropna=False).agg({'payment_status':'count', **basic_agg_columns})

failed_adyen_breakdown1[basic_currency_columns] = failed_adyen_breakdown1[basic_currency_columns].apply(lambda series: series.apply(lambda value: fmt_dnum(value)) )
failed_adyen_breakdown2[basic_currency_columns] = failed_adyen_breakdown2[basic_currency_columns].apply(lambda series: series.apply(lambda value: fmt_dnum(value)) )

missing_refund = adyen_failed2.sort_values(by=['purchase_id']).drop_duplicates(['purchase_id'],keep=False)
missing_refund_subset = pd.DataFrame().assign(tokens=missing_refund['payment_token'], engine_status=missing_refund['Type'], payment_status=missing_refund['payment_status'], amount=missing_refund['payment_total'], method=missing_refund['card_brand'], date=missing_refund['paid_at_mx'], df='only_on_reservamos', purchase_id=missing_refund['purchase_id'], status_check='pending', updated_at=pd.to_datetime('now'))

print(linebr + 'Compras para checar:')
df_counter(missing_refund_subset)

failed_adyen_breakdown1.style if fallidas_en_procesador == 'desglose' else failed_adyen_breakdown2.style

In [ ]:
#@title 5.5 Diferencias en montos cargados entre reporte de Adyen y Reservamos.
#@markdown Diferencias de centavos entre el total del cargo hecho en el reporte de compras y el total del cargo en el reporte del procesador de pago, en transacciones exitosas del mes.
diferencias_en_procesador = "desglose" #@param ["desglose", "registros"]

purchase_adyen_totals = adyen_purchases[['purchase_id', 'purchase_state', 'payment_status', 'payment_engine', 'vendor', 'channel', 'payment_token', 'ticket_count', 'purchase_total', 'payment_total']]

adyen_totals = monthly_adyen_df[['Psp Reference', 'Gross Credit (GC)']]

adyen_totals_df = pd.merge(purchase_adyen_totals, adyen_totals, how='inner', left_on = 'payment_token', right_on = 'Psp Reference')

adyen_totals_df['total_diff'] = adyen_totals_df.payment_total - adyen_totals_df['Gross Credit (GC)']

adyen_totals_subset = adyen_totals_df.query('total_diff != 0')[['purchase_id', 'purchase_state', 'payment_status', 'payment_engine', 'vendor', 'channel', 'payment_token', 'payment_total', 'Gross Credit (GC)', 'total_diff', 'ticket_count']]

adyen_totals_breakdown = adyen_totals_subset.groupby(['purchase_state', 'payment_status'], dropna=False).agg(count=('payment_status','count'), payment_total=('payment_total', 'sum'), adyen_total=('Gross Credit (GC)', 'sum'), total_diff=('total_diff', 'sum'))
adyen_total = adyen_totals_breakdown['adyen_total'].sum()
adyen_payment_total = adyen_totals_breakdown['payment_total'].sum()

adyen_totals_breakdown[['payment_total', 'adyen_total', 'total_diff']] = adyen_totals_breakdown[['payment_total', 'adyen_total', 'total_diff']].apply(lambda series: series.apply(lambda value: fmt_dnum(value)) )

df_counter(adyen_totals_subset)

diff_print(adyen_total, adyen_payment_total)

adyen_totals_breakdown.style if diferencias_en_procesador == 'desglose' else adyen_totals_subset.style

In [ ]:
#@title 5.6 Resumen Adyen
#@markdown

adyen_payment_amount = adyen_purchases['payment_total'].sum()

adyen_total_amount = monthly_adyen_df['Gross Credit (GC)'].sum()

diff_print(adyen_total_amount, adyen_payment_amount)

print(linebr + 'No. de transacciones procesadas por Adyen en el mes: ' + str(monthly_adyen_df.shape[0]) + '  (reporte de adyen)')
adyen_income_on_adyen = monthly_adyen_df["Gross Credit (GC)"].sum()
print('Dinero procesado por Adyen en el mes: ' + "${:0,.2f}".format(adyen_income_on_adyen) + linebr)

print('No. de transacciones procesadas en Reservamos por Adyen en el mes: ' + str(adyen_purchases.shape[0]))
adyen_income_on_admin = adyen_purchases["payment_total"].sum()
print('Dinero procesado por Reservamos a través de Adyen en el mes: ' + "${:0,.2f}".format(adyen_income_on_admin) + linebr)

unsettled_money = adyen_next_month_charges['amount'].sum()
print('No. de transacciones procesadas en Reservamos pendientes en Adyen en el mes: ' + str(adyen_next_month_charges.shape[0]))
print('Dinero procesado por Reservamos que falta procesar en Adyen: ' + "${:0,.2f}".format(unsettled_money) + linebr)

print('No. de transacciones procesadas realmente en Reservamos por Adyen en el mes: ' + str(adyen_purchases.shape[0] - adyen_next_month_charges.shape[0]) + f'  ({adyen_purchases.shape[0]} - {adyen_next_month_charges.shape[0]})')
money_charged_on_reservamos = adyen_income_on_admin - unsettled_money
# Suma de referencias Psp que no aparecen en el reporte de Adyen pero sí en nuestro reporte de compras.
print('Dinero realmente procesado por Reservamos en el mes: ' + "${:0,.2f}".format(money_charged_on_reservamos) + f'  ({"${:0,.2f}".format(adyen_income_on_admin)} - {"${:0,.2f}".format(unsettled_money)})'  + linebr)

failed_payments = adyen_failed.shape[0]
failed_income = adyen_failed["payment_total"].sum()
print('No. total de transacciones fallidas en Reservamos a través de Adyen en el mes: ' + str(failed_payments))
print('Dinero cobrado por Adyen de pagos fallidos en Reservamos: ' + "${:0,.2f}".format(failed_income)  + linebr)

dups_not_included = adyen_dups_not_included['payment_total'].sum()
print('No. de transacciones fallidas en Reservamos a través de Adyen en el mes, no incluidas en el reporte de compras por tener otro pago falllido o exitoso asociado: ' + str(adyen_dups_not_included.shape[0]))
print('Dinero cobrado por Adyen por cobros múltiples fallidos en Reservamos y reembolsado: ' + "${:0,.2f}".format(dups_not_included)  + linebr)

#print('No. de transacciones procesadas en Reservamos por Adyen en el mes menos las fallidas duplicadas asociadas a una compra con otro pago: ' + str(adyen_purchases.shape[0] - adyen_dups_not_included.shape[0]) + '  (reporte de compras filtrado)')
#print('Dinero procesado por Reservamos a través de Adyen en el mes menos las fallidas duplicadas asociadas a una compra con otro pago: ' + "${:0,.2f}".format(adyen_income_on_admin - dups_not_included) + linebr)

money_diff = adyen_income_on_adyen - money_charged_on_reservamos - dups_not_included
print('Diferencia: ' + "${:0,.2f}".format(money_diff) + f'  ({"${:0,.2f}".format(adyen_income_on_adyen)} - ({"${:0,.2f}".format(money_charged_on_reservamos)} + {"${:0,.2f}".format(dups_not_included)}))' + linebr)

adyen_breakdown = all_monthly_adyen_df.groupby(["Type"], dropna=False)[["Gross Credit (GC)", "Gross Debit (GC)"]].sum()
adyen_breakdown[["Gross Credit (GC)", "Gross Debit (GC)"]] = adyen_breakdown[["Gross Credit (GC)", "Gross Debit (GC)"]].apply(lambda series: series.apply(lambda value: f"{value:,}") )
adyen_breakdown.style

# *. MercadoPago

In [ ]:
#@title *.1 Pagos en Reservamos con MercadoPago.
#@markdown Todos los pagos de MercadoPago.
diferencias_openpay = "desglose" #@param ["desglose", "registros"]

# filters only purchases paid by openpay on filtered purchase report.
openpay_purchases = purchases3.query("payment_method == 'mercado_pago'")

df_counter(openpay_purchases)

reservamos_openpay_breakdown= openpay_purchases.groupby(['purchase_state', 'payment_status', 'payment_method'], dropna=False).agg({'payment_method':'count', **basic_agg_columns})
reservamos_openpay_breakdown[basic_currency_columns] = reservamos_openpay_breakdown[basic_currency_columns].apply(lambda series: series.apply(lambda value: fmt_dnum(value)) )
reservamos_openpay_breakdown.style

reservamos_openpay_breakdown.style if diferencias_openpay == 'desglose' else openpay_purchases.style

# 6. Openpay

In [ ]:
#@title 6.1 Pagos en Reservamos con Openpay.
#@markdown Todos los pagos de Paypal en Openpay.
diferencias_openpay = "desglose" #@param ["desglose", "registros"]

# filters only purchases paid by openpay on filtered purchase report.
openpay_purchases = purchases3.query("payment_engine == 'open_pay'")

df_counter(openpay_purchases)

reservamos_openpay_breakdown= openpay_purchases.groupby(['purchase_state', 'payment_status', 'payment_engine'], dropna=False).agg({'payment_engine':'count', **basic_agg_columns})
reservamos_openpay_breakdown[basic_currency_columns] = reservamos_openpay_breakdown[basic_currency_columns].apply(lambda series: series.apply(lambda value: fmt_dnum(value)) )
reservamos_openpay_breakdown.style

reservamos_openpay_breakdown.style if diferencias_openpay == 'desglose' else openpay_purchases.style

In [ ]:
#@title 6.2 Pagos en Openpay.
#@markdown

filtered_openpay_df = openpay_df.query("category == 'charge' and status == 'completed' and order_id == order_id")
filtered_openpay_df2 = filtered_openpay_df[filtered_openpay_df['order_id'].str.contains('RSRBS-')]

df_counter(filtered_openpay_df2)

filtered_openpay_df2['amount'] = filtered_openpay_df2['amount'].replace(regex=',', value='').astype(float)

openpay_breakdown = filtered_openpay_df2.groupby(['category', 'status'], dropna=False).agg({'status':'count','amount':'sum'})
openpay_breakdown[["amount"]] = openpay_breakdown[["amount"]].apply(lambda series: series.apply(lambda value: fmt_dnum(value)) )
openpay_breakdown.style

# to show purchases from breakdown paste 'filtered_openpay_df2.style' at the end of the file.

In [ ]:
#@title 6.3 Diferencias de pagos entre Reservamos y Openpay.
#@markdown Pagos que no aparecen en el reporte de Openpay pero sí en el de Reservamos y viceversa.
diferencias_openpay = "desglose" #@param ["desglose", "registros"]

openpay_tokens = pd.DataFrame().assign(tokens=filtered_openpay_df2['transaction_id'], payment_status=filtered_openpay_df2['status'], amount=filtered_openpay_df2['amount'], method=filtered_openpay_df2['brand'], date=filtered_openpay_df2['creation_date'], df='only_on_openpay')

reservamos_openpay_tokens = pd.DataFrame().assign(tokens=openpay_purchases['payment_token'], payment_status=openpay_purchases['payment_status'], amount=openpay_purchases['payment_total'], method=openpay_purchases['card_brand'], date=openpay_purchases['paid_at_mx'], ticket_count=openpay_purchases['ticket_count'], df='only_on_reservamos', engine='openpay')

openpay_diffs = pd.concat([openpay_tokens, reservamos_openpay_tokens]).drop_duplicates(subset=['tokens'], keep=False)
openpay_diffs_subset = openpay_diffs[['df', 'tokens', 'payment_status', 'amount', 'method', 'date', 'ticket_count', 'engine']].reset_index(drop=True)
df_counter(openpay_diffs_subset)

openpay_next_month_charges = openpay_diffs_subset.query("df == 'only_on_reservamos'")

openpay_diff_breakdown = openpay_diffs_subset.groupby(['df', 'payment_status', 'method'], dropna=False).agg({'method':'count', 'amount': sum})
openpay_diff_breakdown[['amount']] = openpay_diff_breakdown[['amount']].apply(lambda series: series.apply(lambda value: fmt_dnum(value)) )

openpay_diff_breakdown.style if diferencias_openpay == 'desglose' else openpay_diffs_subset.style

In [ ]:
#@title 6.4 Compras fallidas en Reservamos de Openpay.
#@markdown

openpay_failed = purchases3.query("payment_status == 'failed' and payment_engine == 'openpay'")
openpay_failed2 = pd.merge(openpay_failed, openpay_df[['transaction_id','status']], how='left', left_on=['payment_token'], right_on=['transaction_id'])

df_counter(openpay_failed2)

failed_openpay_breakdown = openpay_failed2.groupby(['purchase_state', 'payment_status', 'status'], dropna=False).agg({'payment_status':'count', **basic_agg_columns})
failed_openpay_breakdown[basic_currency_columns] = failed_openpay_breakdown[basic_currency_columns].apply(lambda series: series.apply(lambda value: fmt_dnum(value)) )
failed_openpay_breakdown.style

# to show purchases from breakdown paste 'openpay_failed2.style' at the end of the file.

In [ ]:
#@title 6.5 Diferencias en montos entre reporte de Openpay y Reservamos.
#@markdown Diferencias de centavos entre el total del cargo hecho en el reporte de compras y el total del cargo en el reporte del procesador de pago.
diferencias_en_procesador = "desglose" #@param ["desglose", "registros"]

purchase_openpay_totals = openpay_purchases[['purchase_id', 'purchase_state', 'payment_status', 'payment_engine', 'vendor', 'channel', 'payment_token', 'ticket_count', 'purchase_total', 'payment_total']]

openpay_totals = filtered_openpay_df2[['order_id', 'amount']]

openpay_totals_df = pd.merge(purchase_openpay_totals, openpay_totals, how='inner', left_on = 'payment_token', right_on = 'order_id')

openpay_totals_df['total_diff'] = openpay_totals_df.payment_total - openpay_totals_df['amount']

openpay_totals_subset = openpay_totals_df.query('total_diff != 0')[['purchase_id', 'purchase_state', 'payment_status', 'payment_engine', 'vendor', 'channel', 'payment_token', 'payment_total', 'amount', 'total_diff', 'ticket_count']]

openpay_totals_breakdown = openpay_totals_subset.groupby(['purchase_state', 'payment_status'], dropna=False).agg(count=('payment_status','count'), payment_total=('payment_total', 'sum'), openpay_total=('amount', 'sum'), total_diff=('total_diff', 'sum'))
openpay_total = openpay_totals_breakdown['openpay_total'].sum()
openpay_payment_total = openpay_totals_breakdown['payment_total'].sum()

openpay_totals_breakdown[['payment_total', 'openpay_total', 'total_diff']] = openpay_totals_breakdown[['payment_total', 'openpay_total', 'total_diff']].apply(lambda series: series.apply(lambda value: fmt_dnum(value)) )

df_counter(openpay_totals_subset)
diff_print(openpay_total, openpay_payment_total)

openpay_totals_breakdown.style if diferencias_en_procesador == 'desglose' else openpay_totals_subset.style

In [ ]:
#@title 6.6 Resumen Openpay
#@markdown

openpay_payment_amount = 0 if openpay_purchases.empty else openpay_purchases['payment_total'].sum()

openpay_total_amount = 0 if filtered_openpay_df.empty else filtered_openpay_df2['amount'].sum()

diff_print(openpay_total_amount, openpay_payment_amount)

# 7. Conekta

In [ ]:
#@title 7.1 Pagos en Reservamos con Conekta.
#@markdown Todos los pagos de Paypal en Conekta.
diferencias_conekta = "desglose" #@param ["desglose", "registros"]

# filters only purchases paid by conekta on filtered purchase report.
conekta_purchases = purchases3.query("payment_engine == 'conekta'")

df_counter(conekta_purchases)

reservamos_conekta_breakdown = conekta_purchases.groupby(['purchase_state', 'payment_status', 'payment_engine'], dropna=False).agg({'payment_engine':'count','purchase_total':'sum','payment_total':'sum'})
reservamos_conekta_breakdown[["purchase_total", "payment_total"]] = reservamos_conekta_breakdown[["purchase_total", "payment_total"]].apply(lambda series: series.apply(lambda value: fmt_dnum(value)) )
reservamos_conekta_breakdown.style

reservamos_conekta_breakdown.style if diferencias_conekta == 'desglose' else conekta_purchases.style

In [ ]:
#@title 7.2 Pagos en Conekta.
#@markdown

filtered_conekta_df = conekta_df.query("tipo_transaccion == 'cargo' and estatus == 'paid'")

df_counter(filtered_conekta_df)

conekta_breakdown = filtered_conekta_df.groupby(['tipo_transaccion', 'estatus'], dropna=False).agg({'estatus':'count','monto_cargo':'sum'})
conekta_breakdown[["monto_cargo"]] = conekta_breakdown[["monto_cargo"]].apply(lambda series: series.apply(lambda value: fmt_dnum(value)) )
conekta_breakdown.style


In [ ]:
#@title 7.3 Diferencias de pagos entre Reservamos y Conekta.
#@markdown Pagos que no aparecen en el reporte de Conekta pero sí en el de Reservamos y viceversa.
diferencias_conekta = "registros" #@param ["desglose", "registros"]

conekta_tokens = pd.DataFrame().assign(tokens=filtered_conekta_df['id_orden'], payment_status=filtered_conekta_df['estatus'], amount=filtered_conekta_df['monto_cargo'], method=filtered_conekta_df['metodo_pago'], date=filtered_conekta_df['fecha_pago'], df='only_on_conekta')

reservamos_conekta_tokens = pd.DataFrame().assign(tokens=conekta_purchases['payment_token'], payment_status=conekta_purchases['payment_status'], amount=conekta_purchases['payment_total'], method=conekta_purchases['card_brand'], date=conekta_purchases['paid_at_mx'], ticket_count=conekta_purchases['ticket_count'], df='only_on_reservamos', engine='conekta')

conekta_diffs = pd.concat([conekta_tokens, reservamos_conekta_tokens]).drop_duplicates(subset=['tokens'], keep=False)
conekta_diffs_subset = conekta_diffs[['df', 'tokens', 'payment_status', 'amount', 'method', 'date', 'ticket_count', 'engine']].reset_index(drop=True)
df_counter(conekta_diffs_subset)

conekta_next_month_charges = conekta_diffs_subset.query("df == 'only_on_reservamos'")

conekta_diff_breakdown = conekta_diffs_subset.groupby(['df', 'payment_status', 'method'], dropna=False).agg({'method':'count', 'amount': sum})
conekta_diff_breakdown[['amount']] = conekta_diff_breakdown[['amount']].apply(lambda series: series.apply(lambda value: fmt_dnum(value)) )

conekta_diff_breakdown.style if diferencias_conekta == 'desglose' else conekta_diffs_subset.style

In [ ]:
#@title 7.4 Compras fallidas en Reservamos de Conekta.
#@markdown Probablemente cobradas por el procesador de pagos.

conekta_failed = purchases3.query("payment_status == 'failed' and payment_engine == 'conekta'")
conekta_failed2 = pd.merge(conekta_failed, conekta_df[['id_orden','estatus']], how='left', left_on=['payment_token'], right_on=['id_orden'])

df_counter(conekta_failed2)

failed_conekta_breakdown = conekta_failed2.groupby(['purchase_state', 'payment_status', 'estatus'], dropna=False).agg({'payment_status':'count', **basic_agg_columns})
failed_conekta_breakdown[basic_currency_columns] = failed_conekta_breakdown[basic_currency_columns].apply(lambda series: series.apply(lambda value: fmt_dnum(value)) )
failed_conekta_breakdown.style

# to show purchases from breakdown paste 'conekta_failed2.style' at the end of the file.

In [ ]:
#@title 7.5 Diferencias en montos entre reporte de Conekta y Reservamos.
#@markdown Diferencias de centavos entre el total del cargo hecho en el reporte de compras y el total del cargo en el reporte del procesador de pago.
diferencias_en_procesador = "desglose" #@param ["desglose", "registros"]

purchase_conekta_totals = conekta_purchases[['purchase_id', 'purchase_state', 'payment_status', 'payment_engine', 'vendor', 'channel', 'payment_token', 'ticket_count', 'purchase_total', 'payment_total']]

conekta_totals = filtered_conekta_df[['id_orden', 'monto_cargo']]

conekta_totals_df = pd.merge(purchase_conekta_totals, conekta_totals, how='inner', left_on = 'payment_token', right_on = 'id_orden')

conekta_totals_df['total_diff'] = conekta_totals_df.payment_total - conekta_totals_df['monto_cargo']

conekta_totals_subset = conekta_totals_df.query('total_diff != 0')[['purchase_id', 'purchase_state', 'payment_status', 'payment_engine', 'vendor', 'channel', 'payment_token', 'payment_total', 'monto_cargo', 'total_diff', 'ticket_count']]

conekta_totals_breakdown = conekta_totals_subset.groupby(['purchase_state', 'payment_status'], dropna=False).agg(count=('payment_status','count'), payment_total=('payment_total', 'sum'), conekta_total=('monto_cargo', 'sum'), total_diff=('total_diff', 'sum'))
conekta_total = conekta_totals_breakdown['conekta_total'].sum()
conekta_payment_total = conekta_totals_breakdown['payment_total'].sum()

conekta_totals_breakdown[['payment_total', 'conekta_total', 'total_diff']] = conekta_totals_breakdown[['payment_total', 'conekta_total', 'total_diff']].apply(lambda series: series.apply(lambda value: fmt_dnum(value)) )

df_counter(conekta_totals_subset)
diff_print(conekta_total, conekta_payment_total)

conekta_totals_breakdown.style if diferencias_en_procesador == 'desglose' else conekta_totals_subset.style

In [ ]:
#@title 7.6 Resumen Conekta
#@markdown

conekta_payment_amount = conekta_purchases['payment_total'].sum()

conekta_total_amount = filtered_conekta_df['monto_cargo'].sum()

diff_print(conekta_total_amount, conekta_payment_amount)

# 8. Paypal

In [ ]:
#@title 8.1 Pagos en Reservamos con Paypal.
#@markdown Todos los pagos de Paypal en Reservamos.
diferencias_paypal = "desglose" #@param ["desglose", "registros"]

# filters only purchases paid by Paypal on filtered purchase report.
paypal_purchases = purchases3.query("payment_engine == 'paypal'")

df_counter(paypal_purchases)

reservamos_paypal_breakdown = paypal_purchases.groupby(basic_groupby_columns, dropna=False).agg({'payment_engine':'count', **basic_agg_columns})
reservamos_paypal_breakdown[basic_currency_columns] = reservamos_paypal_breakdown[basic_currency_columns].apply(lambda series: series.apply(lambda value: fmt_dnum(value)) )

reservamos_paypal_breakdown.style if diferencias_paypal == 'desglose' else paypal_purchases.style

In [ ]:
#@title 8.2 Pagos en Paypal.
#@markdown Todos los pagos en el reporte de Paypal.
diferencias_paypal = "desglose" #@param ["desglose", "registros"]

filtered_paypal_df = paypal_df.query("Tipo == 'Pago con Pagos Exprés' and Estado == 'Completado'")
filtered_paypal_df['Bruto'] = filtered_paypal_df['Bruto'].replace(',', '', regex=True).astype(float)

df_counter(filtered_paypal_df)

paypal_breakdown = filtered_paypal_df.groupby(['Tipo', 'Estado'], dropna=False).agg({'Estado':'count','Bruto':'sum'})
paypal_breakdown[["Bruto"]] = paypal_breakdown[["Bruto"]].apply(lambda series: series.apply(lambda value: fmt_dnum(value)) )

paypal_breakdown.style if diferencias_paypal == 'desglose' else filtered_paypal_df.style

In [ ]:
#@title 8.3 Diferencias de pagos entre Reservamos y Paypal.
#@markdown Pagos que no aparecen en el reporte de Paypal pero sí en el de Reservamos y viceversa.
diferencias_paypal = "desglose" #@param ["desglose", "registros"]

paypal_tokens = pd.DataFrame().assign(tokens=filtered_paypal_df['Id. de transacción'], payment_status=filtered_paypal_df['Estado'], amount=filtered_paypal_df['Bruto'], date=filtered_paypal_df['Fecha'], df='only_on_paypal')

reservamos_paypal_tokens = pd.DataFrame().assign(tokens=paypal_purchases['paypal_transaction_id'], payment_status=paypal_purchases['payment_status'], amount=paypal_purchases['payment_total'], date=paypal_purchases['paid_at_mx'], ticket_count=paypal_purchases['ticket_count'], df='only_on_reservamos', engine='paypal')

paypal_diffs = pd.concat([paypal_tokens, reservamos_paypal_tokens]).drop_duplicates(subset=['tokens'], keep=False)
paypal_diffs_subset = paypal_diffs[['df', 'tokens', 'payment_status', 'amount', 'date', 'ticket_count', 'engine']].reset_index(drop=True)
df_counter(paypal_diffs_subset)

paypal_next_month_charges = paypal_diffs_subset.query("df == 'only_on_reservamos'")

paypal_diff_breakdown = paypal_diffs_subset.groupby(['df', 'payment_status'], dropna=False).agg({'payment_status':'count', 'amount': sum})
paypal_diff_breakdown[['amount']] = paypal_diff_breakdown[['amount']].apply(lambda series: series.apply(lambda value: fmt_dnum(value)) )

paypal_diff_breakdown.style if diferencias_paypal == 'desglose' else paypal_diffs_subset.style

In [ ]:
#@title 8.4 Compras fallidas de Paypal en nuestro reporte de compras.
#@markdown
diferencias_paypal = "registros" #@param ["desglose", "registros"]

paypal_failed = purchases3.query("payment_status == 'failed' and payment_engine == 'paypal'")
paypal_failed2 = pd.merge(paypal_failed, paypal_df[['Id. de transacción','Estado']], how='left', left_on=['payment_token'], right_on=['Id. de transacción'])

df_counter(paypal_failed2)

failed_paypal_breakdown = paypal_failed2.groupby(['purchase_state', 'payment_status', 'Estado'], dropna=False).agg({'payment_status':'count', **basic_agg_columns})
failed_paypal_breakdown[basic_currency_columns] = failed_paypal_breakdown[basic_currency_columns].apply(lambda series: series.apply(lambda value: fmt_dnum(value)) )

failed_paypal_breakdown.style if diferencias_paypal == 'desglose' else paypal_failed2.style

In [ ]:
#@title 8.5 Diferencias en montos entre reporte de Paypal y Reservamos.
#@markdown Diferencias de centavos entre el total del cargo hecho en el reporte de compras y el total del cargo en el reporte del procesador de pago.
diferencias_en_procesador = "desglose" #@param ["desglose", "registros"]

purchase_paypal_totals = paypal_purchases[['purchase_id', 'purchase_state', 'payment_status', 'payment_engine', 'vendor', 'channel', 'payment_token', 'ticket_count', 'purchase_total', 'payment_total']]

paypal_totals = filtered_paypal_df[['Id. de transacción', 'Bruto']]

paypal_totals_df = pd.merge(purchase_paypal_totals, paypal_totals, how='inner', left_on = 'payment_token', right_on = 'Id. de transacción')

paypal_totals_df['total_diff'] = paypal_totals_df.payment_total - paypal_totals_df['Bruto']

paypal_totals_subset = paypal_totals_df.query('total_diff != 0')[['purchase_id', 'purchase_state', 'payment_status', 'payment_engine', 'vendor', 'channel', 'payment_token', 'payment_total', 'Bruto', 'total_diff', 'ticket_count']]

paypal_totals_breakdown = paypal_totals_subset.groupby(['purchase_state', 'payment_status'], dropna=False).agg(count=('payment_status','count'), payment_total=('payment_total', 'sum'), paypal_total=('Bruto', 'sum'), total_diff=('total_diff', 'sum'))
paypal_total = paypal_totals_breakdown['paypal_total'].sum()
paypal_payment_total = paypal_totals_breakdown['payment_total'].sum()

paypal_totals_breakdown[['payment_total', 'paypal_total', 'total_diff']] = paypal_totals_breakdown[['payment_total', 'paypal_total', 'total_diff']].apply(lambda series: series.apply(lambda value: fmt_dnum(value)) )

df_counter(paypal_totals_subset)
diff_print(paypal_total, paypal_payment_total)

paypal_totals_breakdown.style if diferencias_en_procesador == 'desglose' else paypal_totals_subset.style

In [ ]:
#@title 8.6 Resumen Paypal
#@markdown

paypal_payment_amount = paypal_purchases['payment_total'].sum()

paypal_total_amount = filtered_paypal_df['Bruto'].sum()

diff_print(paypal_total_amount, paypal_payment_amount)

In [ ]:
#@title Cuentas finales
#purchases4 = purchases3.query("purchase_state in ('completed', 'pending') and payment_status == 'charged'")

purchases5 = purchases3.query("purchase_state == 'completed' and payment_status == 'nopayment'")

money_processed_by_reservamos = purchases3['payment_total'].sum()
money_processed_on_engines = adyen_income_on_adyen + openpay_payment_amount + conekta_payment_amount + paypal_payment_amount
forced_purchases = purchases5['purchase_total'].sum()
actual_income = money_processed_on_engines + forced_purchases

print('Dinero que Reservamos mandó a procesadores de pago: ' + fmt_curr(money_processed_by_reservamos) + "  (compras con pago en el mes)")
print('Dinero efectivo en procesadores de pago: ' + fmt_curr(money_processed_on_engines) + "  (compras con pago en el mes en los procesadores)")
print('Diferencia: ' + fmt_curr(money_processed_by_reservamos - money_processed_on_engines) + linebr)

print('Dinero pendiente por procesar en Adyen: ' + fmt_curr(adyen_payment_amount - adyen_total_amount) + "  (compras que problablemente aparecerán sig mes, checarlas)" + linebr)

print('Ingreso total compras forzadas: ' + fmt_curr(forced_purchases) + "  (compras con estatus completado sin pago en el mes)")
print('Ingreso total: ' + fmt_curr(actual_income) + "  (dinero en procesadores + compras forzadas)" + linebr)

print('TOTALES DE RESERVAMOS, no está cuadrado con las compras pendientes de procesar por los procesadores de pago:')
markup = purchases3['markup'].sum()
nds = purchases3['service_level'].sum()
insurance = purchases3['insurance'].sum()
wallet = purchases3['wallet'].sum()
interests = purchases3['interest'].sum()
comm = purchases3['commission'].sum()
wallet = purchases3['wallet'].sum()
discount = purchases3['discount'].sum()
print('Markup: ' + fmt_curr(markup) + "  (compras con pago en el mes)")
print('NDS: ' + fmt_curr(nds) + "  (compras con pago en el mes)")
print('Seguro: ' + fmt_curr(insurance) + "  (compras con pago en el mes)")
print('Intereses de cargos diferidos: ' + fmt_curr(interests) + "  (compras con pago en el mes)")
print('Comisión: ' + fmt_curr(comm) + "  (compras con pago en el mes)")
print('Descuentos: ' + fmt_curr(discount) + "  (compras con pago en el mes)")
print('Monedero: ' + fmt_curr(discount) + "  (compras con pago en el mes)" + linebr)


# 9. Monedero

In [ ]:
#@title 9.1 Monedero procesado durante el mes

sorted_wallet = wallet_df.sort_values(by=['email', 'created_at'])
sorted_wallet.loc[:, 'year'] = sorted_wallet.created_at.str[0:4]
sorted_wallet.loc[:, 'month'] = sorted_wallet.created_at.str[5:7]

sorted_wallet2 = sorted_wallet.query(f"year == '{año}' and month == '{month(mes)}'")

monthly_wallet_amount_breakdown = sorted_wallet2.groupby(['year', 'month', 'category'], dropna=False).agg(total=('amount', 'sum'))
monthly_wallet_amount_breakdown[['total']] = monthly_wallet_amount_breakdown[['total']].apply(lambda series: series.apply(lambda value: fmt_dnum(value)) )
monthly_wallet_amount_breakdown.style

In [ ]:
#@title 9.2 Top 20 usuarios (lealtad).
#@markdown Usuarios con más puntos sólo acumulados por compras durante el mes.

def sumUp(serie):
  result = 0
  for index, value in serie.iterrows():
    if value['category'] in ['deposit']:
      result += value['amount']
  return result

wallet_df2 = sorted_wallet2.groupby(['email'], dropna=False).apply(lambda x: pd.Series({'cashback': sumUp(x[['amount', 'category']])}))
sorted_wallet3 = wallet_df2.sort_values(by=['cashback'], ascending=False).head(20)

print(sorted_wallet3)

In [ ]:
#@title 9.3 Top 20 usuarios (créditos).
#@markdown Usuarios con más puntos sólo acumulados por reembolsos durante el mes.

def sumUp(serie):
  result = 0
  for index, value in serie.iterrows():
    if value['category'] in ['credit']:
      result += value['amount']
  return result

wallet_df3 = sorted_wallet2.groupby(['email'], dropna=False).apply(lambda x: pd.Series({'crédito': sumUp(x[['amount', 'category']])}))
sorted_wallet4 = wallet_df3.sort_values(by=['crédito'], ascending=False).head(20)

print(sorted_wallet4)

# 10. Resumen

## 10.1 Compras

In [ ]:
#@title 10.1.1 Compras por tipo de compra.
#@markdown Compras que tengan pagos exitosos. <br>

purchas_type_breakdown = income_purchases.groupby(['purchase_state'], dropna=False).agg(transacciones=('vendor','count'), boletos=('ticket_count','sum'), total_compra=('payment_total','sum'))

purchas_type_breakdown.loc['TOTAL']= purchas_type_breakdown.sum(numeric_only=True, axis=0)
purchas_type_breakdown[['transacciones', 'boletos']] = purchas_type_breakdown[['transacciones', 'boletos']].apply(lambda series: series.apply(lambda value: fmt_num(value)) )
purchas_type_breakdown[['total_compra']] = purchas_type_breakdown[['total_compra']].apply(lambda series: series.apply(lambda value: fmt_dnum(value)) )
purchas_type_breakdown.style

# to show purchases from breakdown paste 'purchas_type_df.style' on the next line.

In [ ]:
#@title 10.1.2 Compras por vendedor.
#@markdown Compras que tengan pagos exitosos. <br>

vendor_breakdown = income_purchases.groupby(['vendor'], dropna=False).agg(transacciones=('vendor','count'), boletos=('ticket_count','sum'), total_compra=('payment_total','sum'), total_viaje=('trip_total','sum'))
#vendor_breakdown.loc['column_total']= vendor_breakdown.sum()

vendor_breakdown.loc['TOTAL']= vendor_breakdown.sum(numeric_only=True, axis=0)
vendor_breakdown[['transacciones', 'boletos']] = vendor_breakdown[['transacciones', 'boletos']].apply(lambda series: series.apply(lambda value: fmt_num(value)) )
vendor_breakdown[['total_compra', 'total_viaje']] = vendor_breakdown[['total_compra', 'total_viaje']].apply(lambda series: series.apply(lambda value: fmt_dnum(value)) )
vendor_breakdown.style

# to show purchases from breakdown paste 'income_purchases.style' on the next line.

In [ ]:
#@title 10.1.3 Compras por tipo de transporte.
#@markdown - Separado por bus, avión y mix (mezcla de los anteriores). <br>
#@markdown - Multicarrier bus-bus o avión-avión se cuenta como bus ó avión respectivamente. <br>
#@markdown - Compras con pagos exitosos.

income_purchases.loc[:, 'transportation_type'] = np.where(
    income_purchases['type'].str.contains('AirplanePurchase'), np.where(income_purchases['type'].str.contains('BusPurchase'), 'mix', 'air'), 'bus')

transportation_type_breakdown = income_purchases.groupby(['transportation_type'], dropna=False).agg(transacciones=('purchase_id','count'), boletos=('ticket_count','sum'), total_compra=('payment_total','sum'))

transportation_type_breakdown.loc['TOTAL']= transportation_type_breakdown.sum(numeric_only=True, axis=0)
transportation_type_breakdown[['transacciones', 'boletos']] = transportation_type_breakdown[['transacciones', 'boletos']].apply(lambda series: series.apply(lambda value: fmt_num(value)) )
transportation_type_breakdown[['total_compra']] = transportation_type_breakdown[['total_compra']].apply(lambda series: series.apply(lambda value: fmt_dnum(value)) )
transportation_type_breakdown.style


##10.2 GMV

In [ ]:
#@title 10.2.1.1 GMV Total
#@markdown - Compras con pagos exitosos. <br>
#@markdown - mix se refiere a compra con mix de proveedores, p.e. "Grupo Etn-Turistar,Grupo Flecha Amarilla"

#add transporter columns
airline_ids = [1261]
def find_transporter_name(id):
  if int(id) in airline_ids:
    return 'viva'
  else:
    return bus_transporters_df.loc[bus_transporters_df['id'] == int(id), 'name'].values[0]

income_purchases.loc[:, "uniq_transporter_names"] = income_purchases.transporter.apply(lambda x: find_transporter_name(x) if ',' not in str(x) else ','.join(sorted(set(find_transporter_name(y) for y in(x.split(','))))))
income_purchases.loc[:, "transporter_mix"] = income_purchases.uniq_transporter_names.apply(lambda x: x if ',' not in str(x) else 'mix')

#income_purchases.to_csv(r'income_purchases.csv', header=True)
#files.download('income_purchases.csv')

num_columns = ['transacciones', 'boletos']
dnum_columns = ['total_compra', 'total_viaje', 'nds', 'seguro', 'markup', 'comisión', 'monedero', 'descuentos', 'intereses', 'viajamas']

total_gmv_breakdown = income_purchases.groupby(['transporter_mix'], dropna=False).agg(transacciones=('purchase_id','count'), boletos=('ticket_count','sum'), total_compra=('payment_total','sum'), total_viaje=('trip_total','sum'), nds=('service_level','sum'), seguro=('insurance','sum'), markup=('markup','sum'), comisión=('commission','sum'), monedero=('wallet','sum'), descuentos=('discount','sum'), intereses=('interest','sum'), viajamas=('viajamas','sum'))

total_gmv_breakdown.loc['TOTAL']= total_gmv_breakdown.sum(numeric_only=True, axis=0)
total_gmv_breakdown[num_columns] = total_gmv_breakdown[num_columns].apply(lambda series: series.apply(lambda value: fmt_num(value)) )
total_gmv_breakdown[dnum_columns] = total_gmv_breakdown[dnum_columns].apply(lambda series: series.apply(lambda value: fmt_dnum(value)) )
total_gmv_breakdown.style


In [ ]:
#@title 10.2.1.2 GMV Total por día.
#@markdown - Compras con pagos exitosos. <br>
#@markdown - mix se refiere a compra con mix de proveedores, p.e. "Grupo Etn-Turistar,Grupo Flecha Amarilla"

num_columns = ['transacciones', 'boletos']
dnum_columns = ['total_compra', 'total_viaje', 'nds', 'seguro', 'markup', 'comisión', 'monedero', 'descuentos', 'intereses', 'viajamas']

print(income_purchases['paid_at_mx'].head())

def parse_datetime(value):
    # First, try to parse the value using the format "%Y-%m-%d %H:%M:%S.%f"
    try:
        return pd.to_datetime(value, format='%Y-%m-%d %H:%M:%S.%f')
    except ValueError:
        pass  # If parsing with the first format fails, continue to the next one

    # If parsing with the first format fails, try parsing using the format "%Y-%m-%d %H:%M:%S"
    try:
        return pd.to_datetime(value, format='%Y-%m-%d %H:%M:%S')
    except ValueError:
        # If parsing with both formats fails, return None
        return None

daily_total_gmv_breakdown = income_purchases.groupby(income_purchases['paid_at_mx'].apply(parse_datetime).dt.date).agg(transacciones=('purchase_id','count'), boletos=('ticket_count','sum'), total_compra=('payment_total','sum'), total_viaje=('trip_total','sum'), nds=('service_level','sum'), seguro=('insurance','sum'), markup=('markup','sum'), comisión=('commission','sum'), monedero=('wallet','sum'), descuentos=('discount','sum'), intereses=('interest','sum'), viajamas=('viajamas','sum'))

daily_total_gmv_breakdown.loc['TOTAL']= daily_total_gmv_breakdown.sum(numeric_only=True, axis=0)
daily_total_gmv_breakdown[num_columns] = daily_total_gmv_breakdown[num_columns].apply(lambda series: series.apply(lambda value: fmt_num(value)) )
daily_total_gmv_breakdown[dnum_columns] = daily_total_gmv_breakdown[dnum_columns].apply(lambda series: series.apply(lambda value: fmt_dnum(value)) )
daily_total_gmv_breakdown.style


In [ ]:
#@title 10.2.1.3 GMV Total por día y línea.
#@markdown - Compras con pago exitoso. <br>
#@markdown - mix se refiere a compra con mix de proveedores, p.e. "Grupo Etn-Turistar,Grupo Flecha Amarilla"

daily_total_gmv_breakdown = income_purchases.groupby([income_purchases['paid_at_mx'].apply(parse_datetime).dt.date, 'uniq_transporter_names']).agg(transacciones=('purchase_id','count'), boletos=('ticket_count','sum'), total_compra=('payment_total','sum'), total_viaje=('trip_total','sum'), nds=('service_level','sum'), seguro=('insurance','sum'), markup=('markup','sum'), comisión=('commission','sum'), monedero=('wallet','sum'), descuentos=('discount','sum'), intereses=('interest','sum'), viajamas=('viajamas','sum'))

daily_total_gmv_breakdown[num_columns] = daily_total_gmv_breakdown[num_columns].apply(lambda series: series.apply(lambda value: fmt_num(value)) )
daily_total_gmv_breakdown[dnum_columns] = daily_total_gmv_breakdown[dnum_columns].apply(lambda series: series.apply(lambda value: fmt_dnum(value)) )
daily_total_gmv_breakdown.style

In [ ]:
#@title 10.2.2 GMV Reservamos
#@markdown - Compras con pago exitoso. <br>
#@markdown - mix se refiere a compra con mix de proveedores, p.e. "Grupo Etn-Turistar,Grupo Flecha Amarilla"

reservamos_income_purchases = income_purchases.query("vendor == 'reservamos'")
reservamos_gmv_breakdown = reservamos_income_purchases.groupby(['transporter_mix'], dropna=False).agg(transacciones=('purchase_id','count'), boletos=('ticket_count','sum'), total_compra=('payment_total','sum'), total_viaje=('trip_total','sum'), nds=('service_level','sum'), seguro=('insurance','sum'), markup=('markup','sum'), comisión=('commission','sum'), monedero=('wallet','sum'), descuentos=('discount','sum'), intereses=('interest','sum'), viajamas=('viajamas','sum'))

reservamos_gmv_breakdown.loc['TOTAL']= reservamos_gmv_breakdown.sum(numeric_only=True, axis=0)
reservamos_gmv_breakdown[num_columns] = reservamos_gmv_breakdown[num_columns].apply(lambda series: series.apply(lambda value: fmt_num(value)) )
reservamos_gmv_breakdown[dnum_columns] = reservamos_gmv_breakdown[dnum_columns].apply(lambda series: series.apply(lambda value: fmt_dnum(value)) )
reservamos_gmv_breakdown.style

In [ ]:
#@title 10.2.3 GMV Viva
#@markdown - Compras con pago exitoso. <br>
#@markdown - mix se refiere a compra con mix de proveedores, p.e. "Grupo Etn-Turistar,Grupo Flecha Amarilla"

viva_income_purchases = income_purchases.query("vendor == 'viva'")
viva_gmv_breakdown = viva_income_purchases.groupby(['transporter_mix'], dropna=False).agg(transacciones=('purchase_id','count'), boletos=('ticket_count','sum'), total_compra=('payment_total','sum'), total_viaje=('trip_total','sum'), nds=('service_level','sum'), seguro=('insurance','sum'), markup=('markup','sum'), comisión=('commission','sum'), monedero=('wallet','sum'), descuentos=('discount','sum'), intereses=('interest','sum'), viajamas=('viajamas','sum'))

viva_gmv_breakdown.loc['TOTAL']= viva_gmv_breakdown.sum(numeric_only=True, axis=0)
viva_gmv_breakdown[num_columns] = viva_gmv_breakdown[num_columns].apply(lambda series: series.apply(lambda value: fmt_num(value)) )
viva_gmv_breakdown[dnum_columns] = viva_gmv_breakdown[dnum_columns].apply(lambda series: series.apply(lambda value: fmt_dnum(value)) )
viva_gmv_breakdown.style

In [ ]:
#@title 10.2.4.1 GMV Manuales
#@markdown - Compras con pago exitoso. <br>
#@markdown - mix se refiere a compra con mix de proveedores, p.e. "Grupo Etn-Turistar,Grupo Flecha Amarilla"

manual_income_purchases = income_purchases[income_purchases['uniq_transporter_names'].str.contains('(Grupo ADO|Grupo Senda|Grupo Tap|Grupo Omnibus de México|Grupo Transpaís|Grupo Estrella Blanca|Greyhound)', regex=True)]
manual_gmv_breakdown = manual_income_purchases.groupby(['transporter_mix'], dropna=False).agg(transacciones=('purchase_id','count'), boletos=('ticket_count','sum'), total_compra=('payment_total','sum'), total_viaje=('trip_total','sum'), nds=('service_level','sum'), seguro=('insurance','sum'), markup=('markup','sum'), comisión=('commission','sum'), monedero=('wallet','sum'), descuentos=('discount','sum'), intereses=('interest','sum'), viajamas=('viajamas','sum'))

#print(manual_income_purchases.query("transporter_mix == 'mix'"))
manual_gmv_breakdown.loc['TOTAL']= manual_gmv_breakdown.sum(numeric_only=True, axis=0)
manual_gmv_breakdown[num_columns] = manual_gmv_breakdown[num_columns].apply(lambda series: series.apply(lambda value: fmt_num(value)) )
manual_gmv_breakdown[dnum_columns] = manual_gmv_breakdown[dnum_columns].apply(lambda series: series.apply(lambda value: fmt_dnum(value)) )
manual_gmv_breakdown.style

In [ ]:
#@title 10.2.4.2 GMV Aliados
#@markdown - Compras con pago exitoso. <br>
#@markdown - mix se refiere a compra con mix de proveedores, p.e. "Grupo Etn-Turistar,Grupo Flecha Amarilla"

ally_income_purchases = income_purchases[income_purchases['uniq_transporter_names'].str.contains('(Grupo ADO|Grupo Senda|Grupo Tap|Grupo Omnibus de México|Grupo Transpaís|Grupo Estrella Blanca|Greyhound)') == False]
ally_gmv_breakdown = ally_income_purchases.groupby(['transporter_mix'], dropna=False).agg(transacciones=('purchase_id','count'), boletos=('ticket_count','sum'), total_compra=('payment_total','sum'), total_viaje=('trip_total','sum'), nds=('service_level','sum'), seguro=('insurance','sum'), markup=('markup','sum'), comisión=('commission','sum'), monedero=('wallet','sum'), descuentos=('discount','sum'), intereses=('interest','sum'), viajamas=('viajamas','sum'))

#print(ally_income_purchases.query("transporter_mix == 'mix'"))
ally_gmv_breakdown.loc['TOTAL']= ally_gmv_breakdown.sum(numeric_only=True, axis=0)
ally_gmv_breakdown[num_columns] = ally_gmv_breakdown[num_columns].apply(lambda series: series.apply(lambda value: fmt_num(value)) )
ally_gmv_breakdown[dnum_columns] = ally_gmv_breakdown[dnum_columns].apply(lambda series: series.apply(lambda value: fmt_dnum(value)) )
ally_gmv_breakdown.style

In [ ]:
#@title 10.2.5 GMV Multicarrier mix
#@markdown Compras con pago exitoso.

mix_income_purchases = income_purchases.query("transporter_mix == 'mix'")
mix_gmv_breakdown = mix_income_purchases.groupby(['uniq_transporter_names'], dropna=False).agg(transacciones=('purchase_id','count'), boletos=('ticket_count','sum'), total_compra=('payment_total','sum'), total_viaje=('trip_total','sum'), nds=('service_level','sum'), seguro=('insurance','sum'), markup=('markup','sum'), comisión=('commission','sum'), monedero=('wallet','sum'), descuentos=('discount','sum'), intereses=('interest','sum'), viajamas=('viajamas','sum'))

mix_gmv_breakdown.loc['TOTAL']= mix_gmv_breakdown.sum(numeric_only=True, axis=0)
mix_gmv_breakdown[num_columns] = mix_gmv_breakdown[num_columns].apply(lambda series: series.apply(lambda value: fmt_num(value)) )
mix_gmv_breakdown[dnum_columns] = mix_gmv_breakdown[dnum_columns].apply(lambda series: series.apply(lambda value: fmt_dnum(value)) )
mix_gmv_breakdown.style

In [ ]:
#@title 10.2.6 GMV por procesador
#@markdown - Compras con pagos exitosos. <br>
#@markdown - mix se refiere a compra con mix de proveedores, p.e. "Grupo Etn-Turistar,Grupo Flecha Amarilla"

income_purchases.loc[income_purchases['payment_method'] == 'adyen', ["adyen"]] = income_purchases.payment_total
income_purchases.loc[income_purchases['payment_method'] == 'open_pay', ["open_pay"]] = income_purchases.payment_total
income_purchases.loc[income_purchases['payment_method'] == 'conekta', ["conekta"]] = income_purchases.payment_total
income_purchases.loc[income_purchases['payment_method'] == 'paypal', ["paypal"]] = income_purchases.payment_total
income_purchases.loc[income_purchases['payment_method'] == 'mercado_pago', ["mercado_pago"]] = income_purchases.payment_total


engine_gmv_breakdown = income_purchases.groupby(['transporter_mix']).agg(adyen=('adyen', 'sum'), open_pay=('open_pay', 'sum'), conekta=('conekta', 'sum'), paypal=('paypal', 'sum'), mercado_pago=('mercado_pago', 'sum'))

engine_gmv_breakdown.loc['TOTAL']= engine_gmv_breakdown.sum(numeric_only=True, axis=0)
engine_gmv_breakdown[['adyen', 'open_pay', 'conekta', 'paypal', 'mercado_pago']] = engine_gmv_breakdown[['adyen', 'open_pay', 'conekta', 'paypal', 'mercado_pago']].apply(lambda series: series.apply(lambda value: fmt_dnum(value)) )
engine_gmv_breakdown.style

##10.3 Comisiones

In [ ]:
#@title 10.3.1 Comisiones totales
#@markdown Compras con pago exitoso.

commission_breakdown = income_purchases.groupby(['transporter_mix'], dropna=False).agg(comisión=('commission','sum'))

commission_breakdown.loc['TOTAL']= commission_breakdown.sum(numeric_only=True, axis=0)
commission_breakdown[['comisión']] = commission_breakdown[['comisión']].apply(lambda series: series.apply(lambda value: fmt_dnum(value)) )
commission_breakdown.style

In [ ]:
#@title 10.3.2 Comisiones Reservamos
#@markdown Compras con pago exitoso.

reservamos_commission_breakdown = reservamos_income_purchases.groupby(['transporter_mix'], dropna=False).agg(comisión=('commission','sum'))

reservamos_commission_breakdown.loc['TOTAL']= reservamos_commission_breakdown.sum(numeric_only=True, axis=0)
reservamos_commission_breakdown[['comisión']] = reservamos_commission_breakdown[['comisión']].apply(lambda series: series.apply(lambda value: fmt_dnum(value)) )
reservamos_commission_breakdown.style

In [ ]:
#@title 10.3.3 Comisiones Viva
#@markdown Sólo compras aceptadas.

viva_commission_breakdown = viva_income_purchases.groupby(['transporter_mix'], dropna=False).agg(comisión=('commission','sum'))

viva_commission_breakdown.loc['TOTAL']= viva_commission_breakdown.sum(numeric_only=True, axis=0)
viva_commission_breakdown[['comisión']] = viva_commission_breakdown[['comisión']].apply(lambda series: series.apply(lambda value: fmt_dnum(value)) )
viva_commission_breakdown.style

##10.4 Ancillaries

In [ ]:
#@title 10.4.1 Ancillaries Totales
#@markdown Sólo compras aceptadas.

df_counter(income_purchases)

anc_insurance_breakdown = income_purchases.query("insurance > 0").groupby(lambda _ : 'Seguro').agg(transacciones=('purchase_id','count'), boletos=('ticket_count','sum'), total=('insurance','sum'))
anc_nds_breakdown = income_purchases.query("service_level > 0").groupby(lambda _ : 'Nds').agg(transacciones=('purchase_id','count'), boletos=('ticket_count','sum'), total=('service_level','sum'))
anc_markup_breakdown = income_purchases.query("markup > 0").groupby(lambda _ : 'Markup').agg(transacciones=('purchase_id','count'), boletos=('ticket_count','sum'), total=('markup','sum'))
anc_interest_breakdown = income_purchases.query("interest > 0").groupby(lambda _ : 'Intereses').agg(transacciones=('purchase_id','count'), boletos=('ticket_count','sum'), total=('interest','sum'))
anc_discount_breakdown = income_purchases.query("discount > 0").groupby(lambda _ : 'Descuentos').agg(transacciones=('purchase_id','count'), boletos=('ticket_count','sum'), total=('discount','sum'))
anc_wallet_breakdown = income_purchases.query("wallet > 0").groupby(lambda _ : 'Monedero').agg(transacciones=('purchase_id','count'), boletos=('ticket_count','sum'), total=('wallet','sum'))
anc_viajamas_breakdown = income_purchases.query("viajamas > 0").groupby(lambda _ : 'Viajamas').agg(transacciones=('purchase_id','count'), boletos=('ticket_count','sum'), total=('viajamas','sum'))

ancillaries_breakdown = pd.concat([anc_insurance_breakdown, anc_nds_breakdown, anc_markup_breakdown, anc_interest_breakdown, anc_discount_breakdown, anc_wallet_breakdown, anc_viajamas_breakdown])
ancillaries_breakdown.loc['TOTAL']= ancillaries_breakdown.sum(numeric_only=True, axis=0)
ancillaries_breakdown[num_columns] = ancillaries_breakdown[num_columns].apply(lambda series: series.apply(lambda value: fmt_num(value)) )
ancillaries_breakdown[['total']] = ancillaries_breakdown[['total']].apply(lambda series: series.apply(lambda value: fmt_dnum(value)) )
ancillaries_breakdown.style

In [ ]:
#@title 10.4.2 Ancillaries Totales Bus
#@markdown Sólo compras aceptadas.

anc_bus_df = income_purchases.query("transportation_type == 'bus' ")
df_counter(anc_bus_df)

bus_anc_insurance_breakdown = anc_bus_df.query("insurance > 0").groupby(lambda _ : 'Seguro').agg(transacciones=('purchase_id','count'), boletos=('ticket_count','sum'), total=('insurance','sum'))
bus_anc_nds_breakdown = anc_bus_df.query("service_level > 0").groupby(lambda _ : 'Nds').agg(transacciones=('purchase_id','count'), boletos=('ticket_count','sum'), total=('service_level','sum'))
bus_anc_markup_breakdown = anc_bus_df.query("markup > 0").groupby(lambda _ : 'Markup').agg(transacciones=('purchase_id','count'), boletos=('ticket_count','sum'), total=('markup','sum'))
bus_anc_interest_breakdown = anc_bus_df.query("interest > 0").groupby(lambda _ : 'Intereses').agg(transacciones=('purchase_id','count'), boletos=('ticket_count','sum'), total=('interest','sum'))
bus_anc_discount_breakdown = anc_bus_df.query("discount > 0").groupby(lambda _ : 'Descuentos').agg(transacciones=('purchase_id','count'), boletos=('ticket_count','sum'), total=('discount','sum'))
bus_anc_wallet_breakdown = anc_bus_df.query("wallet > 0").groupby(lambda _ : 'Monedero').agg(transacciones=('purchase_id','count'), boletos=('ticket_count','sum'), total=('wallet','sum'))
bus_anc_viajamas_breakdown = anc_bus_df.query("viajamas > 0").groupby(lambda _ : 'Viajamas').agg(transacciones=('purchase_id','count'), boletos=('ticket_count','sum'), total=('viajamas','sum'))

bus_ancillaries_breakdown = pd.concat([bus_anc_insurance_breakdown, bus_anc_nds_breakdown, bus_anc_markup_breakdown, bus_anc_interest_breakdown, bus_anc_discount_breakdown, bus_anc_wallet_breakdown, bus_anc_viajamas_breakdown])
bus_ancillaries_breakdown.loc['TOTAL']= bus_ancillaries_breakdown.sum(numeric_only=True, axis=0)
bus_ancillaries_breakdown[num_columns] = bus_ancillaries_breakdown[num_columns].apply(lambda series: series.apply(lambda value: fmt_num(value)) )
bus_ancillaries_breakdown[['total']] = bus_ancillaries_breakdown[['total']].apply(lambda series: series.apply(lambda value: fmt_dnum(value)) )
bus_ancillaries_breakdown.style

In [ ]:
#@title 10.4.3 Ancillaries Totales Avión
#@markdown Sólo compras aceptadas.

anc_air_df = income_purchases.query("transportation_type == 'air' ")
df_counter(anc_air_df)

air_anc_insurance_breakdown = anc_air_df.query("insurance > 0").groupby(lambda _ : 'Seguro').agg(transacciones=('purchase_id','count'), boletos=('ticket_count','sum'), total=('insurance','sum'))
air_anc_nds_breakdown = anc_air_df.query("service_level > 0").groupby(lambda _ : 'Nds').agg(transacciones=('purchase_id','count'), boletos=('ticket_count','sum'), total=('service_level','sum'))
air_anc_markup_breakdown = anc_air_df.query("markup > 0").groupby(lambda _ : 'Markup').agg(transacciones=('purchase_id','count'), boletos=('ticket_count','sum'), total=('markup','sum'))
air_anc_interest_breakdown = anc_air_df.query("interest > 0").groupby(lambda _ : 'Intereses').agg(transacciones=('purchase_id','count'), boletos=('ticket_count','sum'), total=('interest','sum'))
air_anc_discount_breakdown = anc_air_df.query("discount > 0").groupby(lambda _ : 'Descuentos').agg(transacciones=('purchase_id','count'), boletos=('ticket_count','sum'), total=('discount','sum'))
air_anc_wallet_breakdown = anc_air_df.query("wallet > 0").groupby(lambda _ : 'Monedero').agg(transacciones=('purchase_id','count'), boletos=('ticket_count','sum'), total=('wallet','sum'))
air_anc_viajamas_breakdown = anc_air_df.query("viajamas > 0").groupby(lambda _ : 'Viajamas').agg(transacciones=('purchase_id','count'), boletos=('ticket_count','sum'), total=('viajamas','sum'))

air_ancillaries_breakdown = pd.concat([air_anc_insurance_breakdown, air_anc_nds_breakdown, air_anc_markup_breakdown, air_anc_interest_breakdown, air_anc_discount_breakdown, air_anc_wallet_breakdown, air_anc_viajamas_breakdown])
air_ancillaries_breakdown.loc['TOTAL']= air_ancillaries_breakdown.sum(numeric_only=True, axis=0)
air_ancillaries_breakdown[num_columns] = air_ancillaries_breakdown[num_columns].apply(lambda series: series.apply(lambda value: fmt_num(value)) )
air_ancillaries_breakdown[['total']] = air_ancillaries_breakdown[['total']].apply(lambda series: series.apply(lambda value: fmt_dnum(value)) )
air_ancillaries_breakdown.style

In [ ]:
#@title 10.4.4 Ancillaries Totales Mix
#@markdown Sólo compras aceptadas.

anc_mix_df = income_purchases.query("transportation_type == 'mix' ")
df_counter(anc_mix_df)

mix_anc_insurance_breakdown = anc_mix_df.query("insurance > 0").groupby(lambda _ : 'Seguro').agg(transacciones=('purchase_id','count'), boletos=('ticket_count','sum'), total=('insurance','sum'))
mix_anc_nds_breakdown = anc_mix_df.query("service_level > 0").groupby(lambda _ : 'Nds').agg(transacciones=('purchase_id','count'), boletos=('ticket_count','sum'), total=('service_level','sum'))
mix_anc_markup_breakdown = anc_mix_df.query("markup > 0").groupby(lambda _ : 'Markup').agg(transacciones=('purchase_id','count'), boletos=('ticket_count','sum'), total=('markup','sum'))
mix_anc_interest_breakdown = anc_mix_df.query("interest > 0").groupby(lambda _ : 'Intereses').agg(transacciones=('purchase_id','count'), boletos=('ticket_count','sum'), total=('interest','sum'))
mix_anc_discount_breakdown = anc_mix_df.query("discount > 0").groupby(lambda _ : 'Descuentos').agg(transacciones=('purchase_id','count'), boletos=('ticket_count','sum'), total=('discount','sum'))
mix_anc_wallet_breakdown = anc_mix_df.query("wallet > 0").groupby(lambda _ : 'Monedero').agg(transacciones=('purchase_id','count'), boletos=('ticket_count','sum'), total=('wallet','sum'))
mix_anc_viajamas_breakdown = anc_mix_df.query("viajamas > 0").groupby(lambda _ : 'Viajamas').agg(transacciones=('purchase_id','count'), boletos=('ticket_count','sum'), total=('viajamas','sum'))

mix_ancillaries_breakdown = pd.concat([mix_anc_insurance_breakdown, mix_anc_nds_breakdown, mix_anc_markup_breakdown, mix_anc_interest_breakdown, mix_anc_discount_breakdown, mix_anc_wallet_breakdown, mix_anc_viajamas_breakdown])
mix_ancillaries_breakdown.loc['TOTAL']= mix_ancillaries_breakdown.sum(numeric_only=True, axis=0)
mix_ancillaries_breakdown[num_columns] = mix_ancillaries_breakdown[num_columns].apply(lambda series: series.apply(lambda value: fmt_num(value)) )
mix_ancillaries_breakdown[['total']] = mix_ancillaries_breakdown[['total']].apply(lambda series: series.apply(lambda value: fmt_dnum(value)) )
mix_ancillaries_breakdown.style

In [ ]:
#@title 10.4.5 Ancillaries Reservamos
#@markdown Sólo compras aceptadas.

df_counter(reservamos_income_purchases)

reservamos_anc_insurance_breakdown = reservamos_income_purchases.query("insurance > 0").groupby(lambda _ : 'Seguro').agg(transacciones=('purchase_id','count'), boletos=('ticket_count','sum'), total=('insurance','sum'))
reservamos_anc_nds_breakdown = reservamos_income_purchases.query("service_level > 0").groupby(lambda _ : 'Nds').agg(transacciones=('purchase_id','count'), boletos=('ticket_count','sum'), total=('service_level','sum'))
reservamos_anc_markup_breakdown = reservamos_income_purchases.query("markup > 0").groupby(lambda _ : 'Markup').agg(transacciones=('purchase_id','count'), boletos=('ticket_count','sum'), total=('markup','sum'))
reservamos_anc_interest_breakdown = reservamos_income_purchases.query("interest > 0").groupby(lambda _ : 'Intereses').agg(transacciones=('purchase_id','count'), boletos=('ticket_count','sum'), total=('interest','sum'))
reservamos_anc_discount_breakdown = reservamos_income_purchases.query("discount > 0").groupby(lambda _ : 'Descuentos').agg(transacciones=('purchase_id','count'), boletos=('ticket_count','sum'), total=('discount','sum'))
reservamos_anc_wallet_breakdown = reservamos_income_purchases.query("wallet > 0").groupby(lambda _ : 'Monedero').agg(transacciones=('purchase_id','count'), boletos=('ticket_count','sum'), total=('wallet','sum'))
reservamos_anc_viajamas_breakdown = reservamos_income_purchases.query("viajamas > 0").groupby(lambda _ : 'Viajamas').agg(transacciones=('purchase_id','count'), boletos=('ticket_count','sum'), total=('viajamas','sum'))

reservamos_ancillaries_breakdown = pd.concat([reservamos_anc_insurance_breakdown, reservamos_anc_nds_breakdown, reservamos_anc_markup_breakdown, reservamos_anc_interest_breakdown, reservamos_anc_discount_breakdown, reservamos_anc_wallet_breakdown, reservamos_anc_viajamas_breakdown])
reservamos_ancillaries_breakdown.loc['TOTAL']= reservamos_ancillaries_breakdown.sum(numeric_only=True, axis=0)
reservamos_ancillaries_breakdown[num_columns] = reservamos_ancillaries_breakdown[num_columns].apply(lambda series: series.apply(lambda value: fmt_num(value)) )
reservamos_ancillaries_breakdown[['total']] = reservamos_ancillaries_breakdown[['total']].apply(lambda series: series.apply(lambda value: fmt_dnum(value)) )
reservamos_ancillaries_breakdown.style

In [ ]:
#@title 10.4.6 Ancillaries Viva
#@markdown Sólo compras aceptadas.

df_counter(viva_income_purchases)

viva_anc_insurance_breakdown = viva_income_purchases.query("insurance > 0").groupby(lambda _ : 'Seguro').agg(transacciones=('purchase_id','count'), boletos=('ticket_count','sum'), total=('insurance','sum'))
viva_anc_nds_breakdown = viva_income_purchases.query("service_level > 0").groupby(lambda _ : 'Nds').agg(transacciones=('purchase_id','count'), boletos=('ticket_count','sum'), total=('service_level','sum'))
viva_anc_markup_breakdown = viva_income_purchases.query("markup > 0").groupby(lambda _ : 'Markup').agg(transacciones=('purchase_id','count'), boletos=('ticket_count','sum'), total=('markup','sum'))
viva_anc_interest_breakdown = viva_income_purchases.query("interest > 0").groupby(lambda _ : 'Intereses').agg(transacciones=('purchase_id','count'), boletos=('ticket_count','sum'), total=('interest','sum'))
viva_anc_discount_breakdown = viva_income_purchases.query("discount > 0").groupby(lambda _ : 'Descuentos').agg(transacciones=('purchase_id','count'), boletos=('ticket_count','sum'), total=('discount','sum'))
viva_anc_wallet_breakdown = viva_income_purchases.query("wallet > 0").groupby(lambda _ : 'Monedero').agg(transacciones=('purchase_id','count'), boletos=('ticket_count','sum'), total=('wallet','sum'))
viva_anc_viajamas_breakdown = viva_income_purchases.query("viajamas > 0").groupby(lambda _ : 'Viajamas').agg(transacciones=('purchase_id','count'), boletos=('ticket_count','sum'), total=('viajamas','sum'))

viva_ancillaries_breakdown = pd.concat([viva_anc_insurance_breakdown, viva_anc_nds_breakdown, viva_anc_markup_breakdown, viva_anc_interest_breakdown, viva_anc_discount_breakdown, viva_anc_wallet_breakdown, viva_anc_viajamas_breakdown])
viva_ancillaries_breakdown.loc['TOTAL']= viva_ancillaries_breakdown.sum(numeric_only=True, axis=0)
viva_ancillaries_breakdown[num_columns] = viva_ancillaries_breakdown[num_columns].apply(lambda series: series.apply(lambda value: fmt_num(value)) )
viva_ancillaries_breakdown[['total']] = viva_ancillaries_breakdown[['total']].apply(lambda series: series.apply(lambda value: fmt_dnum(value)) )
viva_ancillaries_breakdown.style

In [ ]:
#@title 10.4.7 Ancillaries Resumen
#@markdown Sólo compras aceptadas. <br> total_compra de reservamos y viva ya incluyen ancillaries, ¿debería ser total de viaje?

vendor_breakdown2 = income_purchases.groupby(['vendor'], dropna=False).agg(transacciones=('vendor','count'), boletos=('ticket_count','sum'), total=('payment_total','sum'))

res_ancillaries_breakdown = pd.concat([vendor_breakdown2, anc_insurance_breakdown, anc_nds_breakdown, anc_markup_breakdown, anc_interest_breakdown, anc_wallet_breakdown])
res_ancillaries_breakdown.loc['TOTAL']= res_ancillaries_breakdown.sum(numeric_only=True, axis=0)
res_ancillaries_breakdown[num_columns] = res_ancillaries_breakdown[num_columns].apply(lambda series: series.apply(lambda value: fmt_num(value)) )
res_ancillaries_breakdown[['total']] = res_ancillaries_breakdown[['total']].apply(lambda series: series.apply(lambda value: fmt_dnum(value)) )
res_ancillaries_breakdown.style

## 10.5 Procesadores

In [ ]:
#@title 10.5.1 Ventas por procesador en Reservamos.
#@markdown Sólo compras aceptadas. <br>
#@markdown - 'aceptadas' incluye compras completadas, pendientes y contracargos que no tengan pagos fallidos. <br>

engine_income_purchases = income_purchases.query("payment_engine == payment_engine")
engines_breakdown = engine_income_purchases.groupby(['payment_engine'], dropna=False).agg(transacciones=('vendor','count'), boletos=('ticket_count','sum'), total_compra=('payment_total','sum'))

not_engine_income_purchases = income_purchases.query("payment_engine != payment_engine")
print('Compras sin pago: ' + str(fmt_curr(not_engine_income_purchases['payment_total'].sum())))

engines_breakdown.loc['TOTAL']= engines_breakdown.sum(numeric_only=True, axis=0)
engines_breakdown[num_columns] = engines_breakdown[num_columns].apply(lambda series: series.apply(lambda value: fmt_num(value)) )
engines_breakdown[['total_compra']] = engines_breakdown[['total_compra']].apply(lambda series: series.apply(lambda value: fmt_dnum(value)) )
engines_breakdown.style

# to show purchases from breakdown paste 'income_purchases.style' on the next line.

In [ ]:
#@title 10.5.2 Pagos en los procesadores.
#@markdown Pagos settled o exitosos en reportes para Adyen, Openpay, Conekta y Paypal. Total de Reservamos incluye sólo las compras 'aceptadas'.

income_purchases_adyen = income_purchases.query("payment_engine == 'adyen'")
income_purchases_openpay = income_purchases.query("payment_engine == 'openpay'")
income_purchases_conekta = income_purchases.query("payment_engine == 'conekta'")
income_purchases_paypal = income_purchases.query("payment_engine == 'paypal'")

df_data = {'engine': ['adyen', 'openpay', 'conekta', 'paypal'],
           'reservamos_total': [income_purchases_adyen['payment_total'].sum(), income_purchases_openpay['payment_total'].sum(), income_purchases_conekta['payment_total'].sum(), income_purchases_paypal['payment_total'].sum()],
           'engine_total': [monthly_adyen_df['Gross Credit (GC)'].sum(), openpay_total_amount, filtered_conekta_df['monto_cargo'].sum(), filtered_paypal_df['Bruto'].sum()]}
compare_df = pd.DataFrame(data=df_data)

compare_df['diff'] = compare_df['reservamos_total'] - compare_df['engine_total']

pending_breakdown = compare_df.groupby(['engine'], dropna=False).agg(reservamos=('reservamos_total','sum'), procesador=('engine_total', 'sum'), diferencia=('diff', 'sum'))
pending_breakdown.loc['TOTAL']= pending_breakdown.sum(numeric_only=True, axis=0)
pending_breakdown[['reservamos', 'procesador', 'diferencia']] = pending_breakdown[['reservamos', 'procesador', 'diferencia']].apply(lambda series: series.apply(lambda value: fmt_dnum(value)) )
pending_breakdown.style

In [ ]:
#@title 10.5.3 Pagos pendientes en los procesadores.
#@markdown Sólo pagos de Reservamos que deberían aparecer en los reportes de Adyen, Openpay, Conekta y Paypal en los próximos 3 meses. <br> Pueden incluir pagos que entraron el último día del mes pero cuya compra se completó el primer día del mes.

dataframes_for_breakdown = [adyen_next_month_charges, conekta_next_month_charges, paypal_next_month_charges] if openpay_total_amount == 0 else [adyen_next_month_charges, openpay_next_month_charges, conekta_next_month_charges, paypal_next_month_charges]
pending_breakdown_df = pd.concat(dataframes_for_breakdown)

pending_breakdown = pending_breakdown_df.groupby(['engine', 'payment_status'], dropna=False).agg(transacciones=('tokens','count'), boletos=('ticket_count', 'sum'), total_compra=('amount','sum'))
pending_breakdown.loc['TOTAL']= pending_breakdown.sum(numeric_only=True, axis=0)

pending_breakdown[['transacciones', 'boletos']] = pending_breakdown[['transacciones', 'boletos']].apply(lambda series: series.apply(lambda value: fmt_num(value)) )
pending_breakdown[['total_compra']] = pending_breakdown[['total_compra']].apply(lambda series: series.apply(lambda value: fmt_dnum(value)) )
pending_breakdown.style

# Extras (no correr)

In [ ]:
#@title Cargar archivo de pagos no encontrados entre Reservamos y Adyen.

not_found_payments_filename = 'pagos-no-encontrados.csv' #months before
not_found_payments = pd.read_csv(not_found_payments_filename)

In [ ]:
#@title Actualizar archivo de pagos no encontrados este mes entre Reservamos y Adyen.
print('Archivo cargado:')
df_counter(not_found_payments)

# COMPARE ALL PURCHASES AND SAVE IT ON NOT FOUND ON FILE
adyen_tokens0 = pd.DataFrame().assign(tokens=all_monthly_adyen_df['Psp Reference'], engine_status=all_monthly_adyen_df['Type'], amount=all_monthly_adyen_df['Gross Credit (GC)'], method=all_monthly_adyen_df['Payment Method'], date=all_monthly_adyen_df['Creation Date'], df='only_on_adyen', creation_month=all_monthly_adyen_df['Creation Month'])
reservamos_adyen_tokens0 = pd.DataFrame().assign(completed_at_mx=adyen_purchases['completed_at_mx'], tokens=adyen_purchases['payment_token'], payment_status=adyen_purchases['payment_status'], purchase_state=adyen_purchases['purchase_state'], amount=adyen_purchases['payment_total'], method=adyen_purchases['card_brand'], date=adyen_purchases['paid_at_mx'], df='only_on_reservamos', purchase_id=adyen_purchases['purchase_id'], creation_month=adyen_purchases['paid_at_mx'].str[5:7], ticket_count=adyen_purchases['ticket_count'],  engine='adyen')
adyen_diffs0 = pd.concat([adyen_tokens0, reservamos_adyen_tokens0]).drop_duplicates(subset=['tokens'], keep=False)
adyen_diffs_subset0 = adyen_diffs0[['df', 'tokens', 'payment_status', 'amount', 'method', 'date', 'engine_status', 'purchase_id', 'ticket_count', 'engine', 'purchase_state']].reset_index(drop=True)
adyen_diffs_subset0['status_check'] = 'pending'
adyen_diffs_subset0['updated_at'] = pd.to_datetime('now')

differences_subset0 = pd.concat([adyen_diffs_subset0, missing_refund_subset]).drop_duplicates(subset=['tokens'], keep='first')
print(linebr + 'Registros a añadir:')
print(adyen_diffs_subset0.shape[0] + missing_refund_subset.shape[0])

differences_subset = pd.concat([not_found_payments, differences_subset0]).drop_duplicates(subset=['tokens'], keep='first')
print(linebr + 'Archivo actualizado:')
df_counter(differences_subset)

differences_subset.to_csv('pagos-no-encontrados.csv', index=False, encoding='utf-8-sig')

In [ ]:
#@title Cargar archivo de pagos no encontrados entre Reservamos y Adyen.

not_found_payments_filename = 'pagos-no-encontrados.csv' #months before
not_found_payments = pd.read_csv(not_found_payments_filename)

In [ ]:
#@title Buscar, actualizar y descargar pagos de meses anteriores entre Reservamos y Adyen.
#@markdown Transacciones de meses anteriores encontradas este mes.
diferencias_por = "desglose" #@param ["desglose", "registros"]

if (not_found_payments.empty):
  print('Archivo no encontrado o vacío.')
else:
  print('Transacciones en archivo:')
  df_counter(not_found_payments)

  still_not_found_payments = not_found_payments.query("status_check == 'pending'")
  print(linebr + 'Transacciones pendientes de encontrar:')
  df_counter(still_not_found_payments)

  filtered_adyen_df2 = adyen_df.sort_values(by=['Creation Date']).drop_duplicates(['Psp Reference'],keep='last')

  adyen_report_tokens = filtered_adyen_df2['Psp Reference'].values

  not_found_payments.loc[not_found_payments['tokens'].isin(adyen_report_tokens), 'status_check'] = 'found'
  not_found_payments.loc[not_found_payments['tokens'].isin(adyen_report_tokens), 'updated_at'] = pd.to_datetime('now')

  not_found_tokens = not_found_payments['tokens'].values
  for x in not_found_tokens:
    if (x in adyen_report_tokens):
      engine_status = filtered_adyen_df2.loc[filtered_adyen_df2['Psp Reference'] == x, 'Type'].values[0]
      not_found_payments.loc[not_found_payments['tokens'] == x, 'engine_status'] = engine_status

  found_payments = not_found_payments.loc[not_found_payments['tokens'].isin(adyen_report_tokens)]
  print(linebr + 'Transacciones encontradas este mes:')
  df_counter(found_payments)

  checked_payments_total = found_payments.query("engine_status == 'Settled'")['amount'].sum()
  print(linebr + 'Total encontrado y settled este mes:')
  print(fmt_curr(checked_payments_total))

  checked_payments_breakdown = found_payments.groupby(['df', 'status_check', 'payment_status', 'engine_status', 'method'], dropna=False).agg(count=('method','count'), amount=('amount','sum'))
  checked_payments_breakdown[['amount']] = checked_payments_breakdown[['amount']].apply(lambda series: series.apply(lambda value: fmt_dnum(value)) )

  not_found_payments.to_csv(not_found_payments_filename, index=False, encoding='utf-8-sig')
  #files.download(not_found_payments_filename)

  print(linebr + 'Transacciones encontradas:' + linebr)
checked_payments_breakdown.style if diferencias_por == 'desglose' else not_found_payments.style